In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import random
import re

## Contents

**[1. Instalments Payments](#1.-Instalments-Payments)**
    
**[2. Credit Card Balance](#2.-Credit-Card-Balance)**
   
**[3. Bureau & Bureau Balance](#3.-Bureau-&-Bureau-Balance)**

**[4. POS CASH](#4.-POS-CASH)**
    
**[5. Previous Applications](#5.-Previous-Applications)**
   
**[6. Combining everything into train and test set](#6.-Combining-everything-into-train-and-test-set)**
- [6.1. Merging past & bureau data into train_1 and test_1](#6.1.-Merging-past-&-bureau-data-into-train_1-and-test_1)
- [6.2. Merging bureau data into train_2 and test_2](#6.2.-Merging-bureau-data-into-train_2-and-test_2)
- [6.3. Merging past data into train_3 and test_3](#6.3.-Merging-past-data-into-train_3-and-test_3)
- [6.4. Preparing data for train_4 and test_4](#6.4.-Preparing-data-for-train_4-and-test_4)

Looking at other datasets, generate new features:


In [2]:
bureau_bal = pd.read_csv('../datasets/bureau_balance.csv')
bureau = pd.read_csv('../datasets/bureau.csv')
pre_app = pd.read_csv('../datasets/previous_application.csv')
install_pay = pd.read_csv('../datasets/installments_payments.csv')
cc = pd.read_csv('../datasets/credit_card_balance.csv')
pos = pd.read_csv('../datasets/pos_cash_balance.csv')

In [3]:
# reuse our missing data function from EDA noteboook
def missing_data(df):
    total_num_missing = df.isnull().sum()
    total_percent_missing = df.isnull().sum()/len(df)*100
    total_missing_df = pd.concat([total_num_missing, total_percent_missing], axis=1, keys=['Total', '% of Missing Values'])
    total_missing_df = total_missing_df[
        total_missing_df.iloc[:,1] != 0].sort_values('% of Missing Values', ascending=False).round(1)
    print('The dataframe has ' + str(df.shape[1]) + ' columns.\n' 
         'There are ' +str(total_missing_df.shape[0]) + ' columns that have missing values.')
    return total_missing_df

In [4]:
# define function to sum up the columns we want
def do_sum(df, groupby_cols, counted, agg_name):
    gp = df[groupby_cols + [counted]].groupby(groupby_cols)[counted].sum().reset_index().rename(columns={counted: agg_name})
    df = df.merge(gp, on=groupby_cols, how='left')
    return df

In [5]:
# define function to aggregate the df 
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [6]:
# def function to group and merge to a dataframe
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

## 1. Instalments Payments
[top](#Contents)

In [7]:
missing_data(install_pay)

The dataframe has 8 columns.
There are 2 columns that have missing values.


,Total,% of Missing Values
DAYS_ENTRY_PAYMENT,2905,0.0
AMT_PAYMENT,2905,0.0


This dataset gives the history of payments in previous contracts that the current applicant has taken out. We would like to aggregate and engineer some features based on applicant's past payment behavior, specifically, we would like to determine:
- was monthly instalment amount fully paid
- ratio of payment amount and instalment amount, i.e did applicant paid more and hence, aim to end the contract earlier, or was payment less
- was payment on time, early, or late
- if late, was it 30-days or 60-days past due
- recent payment behavior

However, before aggregating the data, we have to appreciate some nuances in applicant's payment behavior. In our example below of a sample contract, we see that payment was made twice for instalment number 3, 10, 11. For these instalment number, the applicant has split the payment twice to meet the instalment amount. However, if we were to do row-wise aggregation, some rows will be flagged as the applicant has underpaid, making him look like a bad customer which is certainly not the case!

In [8]:
# example
install_pay.loc[install_pay['SK_ID_PREV'] == 2069908].sort_values('NUM_INSTALMENT_NUMBER')

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
2498001,2069908,166397,1.0,1,-1010.0,-1025.0,27982.575,27982.575
2822705,2069908,166397,1.0,2,-980.0,-997.0,27982.575,27982.575
3557451,2069908,166397,1.0,3,-950.0,-968.0,27982.575,27982.035
1251429,2069908,166397,1.0,3,-950.0,-937.0,27982.575,0.540
95318,2069908,166397,1.0,4,-920.0,-937.0,27982.575,27982.575
2283016,2069908,166397,1.0,5,-890.0,-906.0,27982.575,27982.575
2578653,2069908,166397,1.0,6,-860.0,-877.0,27982.575,27982.575
3770650,2069908,166397,1.0,7,-830.0,-843.0,27982.575,27982.575
1224304,2069908,166397,1.0,8,-800.0,-806.0,27982.575,27982.575
1159113,2069908,166397,1.0,9,-770.0,-772.0,27982.575,27982.575


In [9]:
install_pay = do_sum(install_pay, ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], 'AMT_PAYMENT', 'AMT_PAYMENT_GROUPED')

In [10]:
# check
install_pay.loc[install_pay['SK_ID_PREV'] == 2069908].sort_values('NUM_INSTALMENT_NUMBER')


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,AMT_PAYMENT_GROUPED
2498001,2069908,166397,1.0,1,-1010.0,-1025.0,27982.575,27982.575,27982.575
2822705,2069908,166397,1.0,2,-980.0,-997.0,27982.575,27982.575,27982.575
3557451,2069908,166397,1.0,3,-950.0,-968.0,27982.575,27982.035,27982.575
1251429,2069908,166397,1.0,3,-950.0,-937.0,27982.575,0.540,27982.575
95318,2069908,166397,1.0,4,-920.0,-937.0,27982.575,27982.575,27982.575
2283016,2069908,166397,1.0,5,-890.0,-906.0,27982.575,27982.575,27982.575
2578653,2069908,166397,1.0,6,-860.0,-877.0,27982.575,27982.575,27982.575
3770650,2069908,166397,1.0,7,-830.0,-843.0,27982.575,27982.575,27982.575
1224304,2069908,166397,1.0,8,-800.0,-806.0,27982.575,27982.575,27982.575
1159113,2069908,166397,1.0,9,-770.0,-772.0,27982.575,27982.575,27982.575


In [11]:
# create features that calculate number of days payment was early or late
install_pay['DAYS_BEFORE_DUE'] = install_pay['DAYS_INSTALMENT'] - install_pay['DAYS_ENTRY_PAYMENT']
install_pay['DAYS_BEFORE_DUE'] = install_pay['DAYS_BEFORE_DUE'].apply(lambda x: 0 if x<=0 else x)
install_pay['DAYS_PAST_DUE'] =  install_pay['DAYS_ENTRY_PAYMENT'] - install_pay['DAYS_INSTALMENT'] 
install_pay['DAYS_PAST_DUE'] = install_pay['DAYS_PAST_DUE'].apply(lambda x: 0 if x<=0 else x)

# flag late payments
install_pay['FLAG_LATE_PAY'] = install_pay['DAYS_PAST_DUE'].apply(lambda x: 1 if x>0 else 0)

# % of payments that were late
install_pay['INSTALL_PAY_RATIO'] = install_pay['AMT_PAYMENT'] / install_pay['AMT_INSTALMENT']
install_pay['LATE_PAY_RATIO'] = install_pay.apply(lambda x: x['INSTALL_PAY_RATIO'] if x['FLAG_LATE_PAY']==1 else 0, axis=1)

# Flag late payments that have a significant amount > 20%
install_pay['SIGNIFICANT_LATE_PAY'] = install_pay['LATE_PAY_RATIO'].apply(lambda x: 1 if x > 0.20 else 0)

# create feature FLAG_30DAYS_DUE where 1 = payment was late by 30 days
install_pay['FLAG_15DAYS_DUE'] = install_pay['DAYS_PAST_DUE'].apply(lambda x: 1 if x >= 15 else 0)

# create feature FLAG_30DAYS_DUE where 1 = payment was late by 30 days
install_pay['FLAG_30DAYS_DUE'] = install_pay['DAYS_PAST_DUE'].apply(lambda x: 1 if x >= 30 else 0)

# create feature FLAG_60DAYS_DUE where 1 = payment was late by 60 days
install_pay['FLAG_60DAYS_DUE'] = install_pay['DAYS_PAST_DUE'].apply(lambda x: 1 if x >= 60 else 0)

# create feature PAYMENT_RATIO which is payment amt / instalment amt
install_pay['PAYMENT_RATIO'] = install_pay['AMT_PAYMENT_GROUPED'] / install_pay['AMT_INSTALMENT']

# create feature FLAG_PAYMENT_OVER which indicates that payment was more than the instalment amount
install_pay['PAYMENT_OVER'] = install_pay['AMT_PAYMENT'] - install_pay['AMT_INSTALMENT']
install_pay['FLAG_PAYMENT_OVER'] = install_pay['PAYMENT_OVER'].apply(lambda x: 1 if x>0 else 0)


In [12]:
# for overall instalment history
instalments_agg = {'SK_ID_PREV': ['nunique'],
                    'DAYS_ENTRY_PAYMENT': ['mean', 'min', 'max'], 
                    'AMT_INSTALMENT': ['mean', 'min', 'max', 'std'], 
                    'AMT_PAYMENT': ['mean', 'min', 'max', 'std'],
                    'DAYS_BEFORE_DUE': ['mean', 'max', 'std'],
                    'DAYS_PAST_DUE': ['mean', 'max', 'std'],
                    'FLAG_LATE_PAY': ['mean', 'sum'],
                    'INSTALL_PAY_RATIO': ['mean', 'std'],
                    'LATE_PAY_RATIO': ['mean', 'std'],
                    'SIGNIFICANT_LATE_PAY': ['mean', 'sum'], 
                    'FLAG_15DAYS_DUE': ['mean'], 
                    'FLAG_30DAYS_DUE': ['mean'],
                    'FLAG_60DAYS_DUE': ['mean'],
                    'PAYMENT_RATIO': ['mean'],
                    'FLAG_PAYMENT_OVER': ['mean']
                    }

# for recent instalment history
instalments_time_agg = {'SK_ID_PREV': ['size'],
                        'DAYS_ENTRY_PAYMENT': ['mean', 'min', 'max'], 
                        'AMT_INSTALMENT': ['mean', 'min', 'max', 'std'], 
                        'AMT_PAYMENT': ['mean', 'min', 'max', 'std'],
                        'DAYS_BEFORE_DUE': ['mean', 'max', 'std'],
                        'DAYS_PAST_DUE': ['mean', 'max', 'std'],
                        'FLAG_LATE_PAY': ['mean', 'sum'],
                        'INSTALL_PAY_RATIO': ['mean', 'std'],
                        'LATE_PAY_RATIO': ['mean', 'std'],
                        'SIGNIFICANT_LATE_PAY': ['mean', 'sum'], 
                        'FLAG_15DAYS_DUE': ['mean'], 
                        'FLAG_30DAYS_DUE': ['mean'],
                        'FLAG_60DAYS_DUE': ['mean'],
                        'PAYMENT_RATIO': ['mean'],
                        'FLAG_PAYMENT_OVER': ['mean']
                        }

In [13]:
# aggregate df
install_pay_agg = group(install_pay, 'INS_', instalments_agg)

In [14]:
install_pay_agg.head(3)

,SK_ID_CURR,INS_SK_ID_PREV_NUNIQUE,INS_DAYS_ENTRY_PAYMENT_MEAN,INS_DAYS_ENTRY_PAYMENT_MIN,INS_DAYS_ENTRY_PAYMENT_MAX,INS_AMT_INSTALMENT_MEAN,INS_AMT_INSTALMENT_MIN,INS_AMT_INSTALMENT_MAX,INS_AMT_INSTALMENT_STD,INS_AMT_PAYMENT_MEAN,INS_AMT_PAYMENT_MIN,INS_AMT_PAYMENT_MAX,INS_AMT_PAYMENT_STD,INS_DAYS_BEFORE_DUE_MEAN,INS_DAYS_BEFORE_DUE_MAX,INS_DAYS_BEFORE_DUE_STD,INS_DAYS_PAST_DUE_MEAN,INS_DAYS_PAST_DUE_MAX,INS_DAYS_PAST_DUE_STD,INS_FLAG_LATE_PAY_MEAN,INS_FLAG_LATE_PAY_SUM,INS_INSTALL_PAY_RATIO_MEAN,INS_INSTALL_PAY_RATIO_STD,INS_LATE_PAY_RATIO_MEAN,INS_LATE_PAY_RATIO_STD,INS_SIGNIFICANT_LATE_PAY_MEAN,INS_SIGNIFICANT_LATE_PAY_SUM,INS_FLAG_15DAYS_DUE_MEAN,INS_FLAG_30DAYS_DUE_MEAN,INS_FLAG_60DAYS_DUE_MEAN,INS_PAYMENT_RATIO_MEAN,INS_FLAG_PAYMENT_OVER_MEAN
0,100001,2,-2195.000000,-2916.0,-1628.0,5885.132143,3951.000,17397.900,5076.676624,5885.132143,3951.000,17397.900,5076.676624,8.857143,36.0,12.811825,1.571429,11.0,4.157609,0.142857,1,1.000000,0.000000,0.142857,0.377964,0.142857,1,0.0,0.0,0.0,1.000000,0.000000
1,100002,1,-315.421053,-587.0,-49.0,11559.247105,9251.775,53093.745,10058.037722,11559.247105,9251.775,53093.745,10058.037722,20.421053,31.0,4.925171,0.000000,0.0,0.000000,0.000000,0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,1.000000,0.000000
2,100003,3,-1385.320000,-2324.0,-544.0,64754.586000,6662.970,560835.360,110542.592300,64754.586000,6662.970,560835.360,110542.592300,7.160000,14.0,3.726929,0.000000,0.0,0.000000,0.000000,0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,1.000000,0.000000
3,100004,1,-761.666667,-795.0,-727.0,7096.155000,5357.250,10573.965,3011.871810,7096.155000,5357.250,10573.965,3011.871810,7.666667,11.0,4.163332,0.000000,0.0,0.000000,0.000000,0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,1.000000,0.000000
4,100005,1,-609.555556,-736.0,-470.0,6240.205000,4813.200,17656.245,4281.015000,6240.205000,4813.200,17656.245,4281.015000,23.666667,37.0,13.285330,0.111111,1.0,0.333333,0.111111,1,1.000000,0.000000,0.111111,0.333333,0.111111,1,0.0,0.0,0.0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339582,456251,1,-156.285714,-237.0,-38.0,7492.924286,6605.910,12815.010,2346.819209,7492.924286,6605.910,12815.010,2346.819209,36.285714,46.0,14.079705,0.000000,0.0,0.000000,0.000000,0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,1.000000,0.000000
339583,456252,1,-2393.833333,-2470.0,-2327.0,10069.867500,10046.880,10074.465,11.261529,10069.867500,10046.880,10074.465,11.261529,3.333333,11.0,4.082483,0.500000,3.0,1.224745,0.166667,1,1.000000,0.000000,0.166667,0.408248,0.166667,1,0.0,0.0,0.0,1.000000,0.000000
339584,456253,3,-2387.428571,-2915.0,-1738.0,4399.707857,2754.450,5575.185,1145.446623,4115.915357,27.270,5575.185,1638.228743,15.142857,51.0,15.629642,0.642857,9.0,2.405351,0.071429,1,0.928571,0.265292,0.070938,0.265427,0.071429,1,0.0,0.0,0.0,1.000000,0.000000
339585,456254,2,-161.263158,-317.0,-18.0,10239.832895,2296.440,19065.825,8602.512994,10239.832895,2296.440,19065.825,8602.512994,19.000000,31.0,8.198916,0.000000,0.0,0.000000,0.000000,0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,1.000000,0.000000


In [15]:
# looking at data in recent 2, and 4 years
for months in [24, 48]:
    recent_id_prev = install_pay[install_pay['DAYS_INSTALMENT'] >= -30*months]['SK_ID_PREV'].unique()
    pay_recent = install_pay[install_pay['SK_ID_PREV'].isin(recent_id_prev)]
    prefix = f'INS_{months}M_'
    install_pay_agg = group_and_merge(pay_recent, install_pay_agg, prefix, instalments_time_agg)

In [16]:
# check
install_pay_agg.head(3)

,SK_ID_CURR,INS_SK_ID_PREV_NUNIQUE,INS_DAYS_ENTRY_PAYMENT_MEAN,INS_DAYS_ENTRY_PAYMENT_MIN,INS_DAYS_ENTRY_PAYMENT_MAX,INS_AMT_INSTALMENT_MEAN,INS_AMT_INSTALMENT_MIN,INS_AMT_INSTALMENT_MAX,INS_AMT_INSTALMENT_STD,INS_AMT_PAYMENT_MEAN,INS_AMT_PAYMENT_MIN,INS_AMT_PAYMENT_MAX,INS_AMT_PAYMENT_STD,INS_DAYS_BEFORE_DUE_MEAN,INS_DAYS_BEFORE_DUE_MAX,INS_DAYS_BEFORE_DUE_STD,INS_DAYS_PAST_DUE_MEAN,INS_DAYS_PAST_DUE_MAX,INS_DAYS_PAST_DUE_STD,INS_FLAG_LATE_PAY_MEAN,INS_FLAG_LATE_PAY_SUM,INS_INSTALL_PAY_RATIO_MEAN,INS_INSTALL_PAY_RATIO_STD,INS_LATE_PAY_RATIO_MEAN,INS_LATE_PAY_RATIO_STD,INS_SIGNIFICANT_LATE_PAY_MEAN,INS_SIGNIFICANT_LATE_PAY_SUM,INS_FLAG_15DAYS_DUE_MEAN,INS_FLAG_30DAYS_DUE_MEAN,INS_FLAG_60DAYS_DUE_MEAN,INS_PAYMENT_RATIO_MEAN,INS_FLAG_PAYMENT_OVER_MEAN,INS_24M_SK_ID_PREV_SIZE,INS_24M_DAYS_ENTRY_PAYMENT_MEAN,INS_24M_DAYS_ENTRY_PAYMENT_MIN,INS_24M_DAYS_ENTRY_PAYMENT_MAX,INS_24M_AMT_INSTALMENT_MEAN,INS_24M_AMT_INSTALMENT_MIN,INS_24M_AMT_INSTALMENT_MAX,INS_24M_AMT_INSTALMENT_STD,INS_24M_AMT_PAYMENT_MEAN,INS_24M_AMT_PAYMENT_MIN,INS_24M_AMT_PAYMENT_MAX,INS_24M_AMT_PAYMENT_STD,INS_24M_DAYS_BEFORE_DUE_MEAN,INS_24M_DAYS_BEFORE_DUE_MAX,INS_24M_DAYS_BEFORE_DUE_STD,INS_24M_DAYS_PAST_DUE_MEAN,INS_24M_DAYS_PAST_DUE_MAX,INS_24M_DAYS_PAST_DUE_STD,INS_24M_FLAG_LATE_PAY_MEAN,INS_24M_FLAG_LATE_PAY_SUM,INS_24M_INSTALL_PAY_RATIO_MEAN,INS_24M_INSTALL_PAY_RATIO_STD,INS_24M_LATE_PAY_RATIO_MEAN,INS_24M_LATE_PAY_RATIO_STD,INS_24M_SIGNIFICANT_LATE_PAY_MEAN,INS_24M_SIGNIFICANT_LATE_PAY_SUM,INS_24M_FLAG_15DAYS_DUE_MEAN,INS_24M_FLAG_30DAYS_DUE_MEAN,INS_24M_FLAG_60DAYS_DUE_MEAN,INS_24M_PAYMENT_RATIO_MEAN,INS_24M_FLAG_PAYMENT_OVER_MEAN,INS_48M_SK_ID_PREV_SIZE,INS_48M_DAYS_ENTRY_PAYMENT_MEAN,INS_48M_DAYS_ENTRY_PAYMENT_MIN,INS_48M_DAYS_ENTRY_PAYMENT_MAX,INS_48M_AMT_INSTALMENT_MEAN,INS_48M_AMT_INSTALMENT_MIN,INS_48M_AMT_INSTALMENT_MAX,INS_48M_AMT_INSTALMENT_STD,INS_48M_AMT_PAYMENT_MEAN,INS_48M_AMT_PAYMENT_MIN,INS_48M_AMT_PAYMENT_MAX,INS_48M_AMT_PAYMENT_STD,INS_48M_DAYS_BEFORE_DUE_MEAN,INS_48M_DAYS_BEFORE_DUE_MAX,INS_48M_DAYS_BEFORE_DUE_STD,INS_48M_DAYS_PAST_DUE_MEAN,INS_48M_DAYS_PAST_DUE_MAX,INS_48M_DAYS_PAST_DUE_STD,INS_48M_FLAG_LATE_PAY_MEAN,INS_48M_FLAG_LATE_PAY_SUM,INS_48M_INSTALL_PAY_RATIO_MEAN,INS_48M_INSTALL_PAY_RATIO_STD,INS_48M_LATE_PAY_RATIO_MEAN,INS_48M_LATE_PAY_RATIO_STD,INS_48M_SIGNIFICANT_LATE_PAY_MEAN,INS_48M_SIGNIFICANT_LATE_PAY_SUM,INS_48M_FLAG_15DAYS_DUE_MEAN,INS_48M_FLAG_30DAYS_DUE_MEAN,INS_48M_FLAG_60DAYS_DUE_MEAN,INS_48M_PAYMENT_RATIO_MEAN,INS_48M_FLAG_PAYMENT_OVER_MEAN
0,100001,2,-2195.000000,-2916.0,-1628.0,5885.132143,3951.000,17397.900,5076.676624,5885.132143,3951.000,17397.900,5076.676624,8.857143,36.0,12.811825,1.571429,11.0,4.157609,0.142857,1,1.000000,0.000000,0.142857,0.377964,0.142857,1,0.0,0.0,0.0,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,1,-315.421053,-587.0,-49.0,11559.247105,9251.775,53093.745,10058.037722,11559.247105,9251.775,53093.745,10058.037722,20.421053,31.0,4.925171,0.000000,0.0,0.000000,0.000000,0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,1.000000,0.000000,19.0,-315.421053,-587.0,-49.0,11559.247105,9251.775,53093.745,10058.037722,11559.247105,9251.775,53093.745,10058.037722,20.421053,31.0,4.925171,0.000000,0.0,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,19.0,-315.421053,-587.0,-49.0,11559.247105,9251.775,53093.745,10058.037722,11559.247105,9251.775,53093.745,10058.037722,20.421053,31.0,4.925171,0.000000,0.0,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000
2,100003,3,-1385.320000,-2324.0,-544.0,64754.586000,6662.970,560835.360,110542.592300,64754.586000,6662.970,560835.360,110542.592300,7.160000,14.0,3.726929,0.000000,0.0,0.000000,0.000000,0,1.000000,0.000000,0.000000,0

## 2. Credit Card Balance
[top](#Contents)

In [17]:
missing_data(cc)

The dataframe has 23 columns.
There are 9 columns that have missing values.


,Total,% of Missing Values
AMT_PAYMENT_CURRENT,767988,20.0
AMT_DRAWINGS_ATM_CURRENT,749816,19.5
AMT_DRAWINGS_OTHER_CURRENT,749816,19.5
AMT_DRAWINGS_POS_CURRENT,749816,19.5
CNT_DRAWINGS_ATM_CURRENT,749816,19.5
CNT_DRAWINGS_OTHER_CURRENT,749816,19.5
CNT_DRAWINGS_POS_CURRENT,749816,19.5
AMT_INST_MIN_REGULARITY,305236,7.9
CNT_INSTALMENT_MATURE_CUM,305236,7.9


In [18]:
# sort df by SK_ID_PREV and Months Balance to order the data
cc = cc.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])

In [19]:
# example of a previous credit card account
cc.loc[cc['SK_ID_PREV']==1000096]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
1514052,1000096,306118,-96,177072.570,180000,13500.0,13500.000,0.0,0.000,9000.000,9000.000,9000.000,171077.805,177072.570,177072.570,1.0,1,0.0,0.0,7.0,Active,0,0
96202,1000096,306118,-95,178592.940,180000,4500.0,4500.000,0.0,0.000,9000.000,9000.000,9000.000,172572.570,178592.940,178592.940,1.0,1,0.0,0.0,8.0,Active,0,0
2250768,1000096,306118,-94,180242.370,180000,0.0,5098.500,0.0,5098.500,9000.000,9000.000,9000.000,174691.440,180242.370,180242.370,0.0,1,0.0,1.0,9.0,Active,0,0
734483,1000096,306118,-93,176652.495,180000,0.0,0.000,0.0,0.000,9000.000,9000.000,9000.000,171242.370,176652.495,176652.495,0.0,0,0.0,0.0,10.0,Active,0,0
2117399,1000096,306118,-92,31753.710,180000,0.0,0.000,0.0,0.000,9000.000,148500.000,148500.000,28152.495,31753.710,31753.710,0.0,0,0.0,0.0,11.0,Active,0,0
1899634,1000096,306118,-91,25857.450,180000,0.0,0.000,0.0,0.000,9000.000,9000.000,9000.000,22753.710,25857.450,25857.450,0.0,0,0.0,0.0,12.0,Active,0,0
911548,1000096,306118,-90,19906.650,180000,0.0,0.000,0.0,0.000,9000.000,9000.000,9000.000,16857.450,19906.650,19906.650,0.0,0,0.0,0.0,13.0,Active,0,0
1342104,1000096,306118,-89,21766.410,180000,6750.0,7200.000,0.0,450.000,9000.000,9000.000,9000.000,18106.650,21766.410,21766.410,1.0,2,0.0,1.0,14.0,Active,0,0
1033852,1000096,306118,-88,79502.985,180000,0.0,63027.000,0.0,63027.000,9000.000,9000.000,9000.000,75793.410,79502.985,79502.985,0.0,1,0.0,1.0,15.0,Active,0,0
1906048,1000096,306118,-87,119878.740,180000,45000.0,45000.000,0.0,0.000,9000.000,9000.000,9000.000,115502.985,119878.740,119878.740,2.0,2,0.0,0.0,16.0,Active,0,0


In [20]:
cc.shape

(3840312, 23)

In [21]:
# set marker column to identify the row with contract status == 'Completed'
cc['marker'] = cc['NAME_CONTRACT_STATUS'].apply(lambda x: -99 if x=='Completed' else random.uniform(0,100))

#drop duplicates based on subsets
cc.drop_duplicates(subset=['SK_ID_PREV', 'NAME_CONTRACT_STATUS', 'marker'], keep='first', inplace=True)


In [22]:
# check
# example of a previous credit card account
cc.loc[cc['SK_ID_PREV']==1000096]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,marker
1514052,1000096,306118,-96,177072.570,180000,13500.0,13500.000,0.0,0.000,9000.000,9000.000,9000.000,171077.805,177072.570,177072.570,1.0,1,0.0,0.0,7.0,Active,0,0,80.153613
96202,1000096,306118,-95,178592.940,180000,4500.0,4500.000,0.0,0.000,9000.000,9000.000,9000.000,172572.570,178592.940,178592.940,1.0,1,0.0,0.0,8.0,Active,0,0,40.130008
2250768,1000096,306118,-94,180242.370,180000,0.0,5098.500,0.0,5098.500,9000.000,9000.000,9000.000,174691.440,180242.370,180242.370,0.0,1,0.0,1.0,9.0,Active,0,0,45.350077
734483,1000096,306118,-93,176652.495,180000,0.0,0.000,0.0,0.000,9000.000,9000.000,9000.000,171242.370,176652.495,176652.495,0.0,0,0.0,0.0,10.0,Active,0,0,31.532693
2117399,1000096,306118,-92,31753.710,180000,0.0,0.000,0.0,0.000,9000.000,148500.000,148500.000,28152.495,31753.710,31753.710,0.0,0,0.0,0.0,11.0,Active,0,0,90.145127
1899634,1000096,306118,-91,25857.450,180000,0.0,0.000,0.0,0.000,9000.000,9000.000,9000.000,22753.710,25857.450,25857.450,0.0,0,0.0,0.0,12.0,Active,0,0,24.191556
911548,1000096,306118,-90,19906.650,180000,0.0,0.000,0.0,0.000,9000.000,9000.000,9000.000,16857.450,19906.650,19906.650,0.0,0,0.0,0.0,13.0,Active,0,0,99.571806
1342104,1000096,306118,-89,21766.410,180000,6750.0,7200.000,0.0,450.000,9000.000,9000.000,9000.000,18106.650,21766.410,21766.410,1.0,2,0.0,1.0,14.0,Active,0,0,40.230359
1033852,1000096,306118,-88,79502.985,180000,0.0,63027.000,0.0,63027.000,9000.000,9000.000,9000.000,75793.410,79502.985,79502.985,0.0,1,0.0,1.0,15.0,Active,0,0,42.544986
1906048,1000096,306118,-87,119878.740,180000,45000.0,45000.000,0.0,0.000,9000.000,9000.000,9000.000,115502.985,119878.740,119878.740,2.0,2,0.0,0.0,16.0,Active,0,0,68.513159


In [23]:
# create feature to look at usage to credit limit
cc['USAGE_RATIO'] = cc['AMT_BALANCE'] / cc['AMT_CREDIT_LIMIT_ACTUAL']

# feature to look at payment rate on min payment
cc['PAYMENT_RATE'] = cc['AMT_PAYMENT_CURRENT']/cc['AMT_INST_MIN_REGULARITY']

# feature to look at drawing limit rate
cc['DRAWING_LIMIT_RATE'] = cc['AMT_DRAWINGS_ATM_CURRENT'] / cc['AMT_CREDIT_LIMIT_ACTUAL']

# create feature FLAG_30DAYS_DUE where 1 = payment was late by 15 days
cc['FLAG_15DAYS_DUE'] = cc['SK_DPD'].apply(lambda x: 1 if x >= 15 else 0)

# create feature FLAG_30DAYS_DUE where 1 = payment was late by 30 days
cc['FLAG_30DAYS_DUE'] = cc['SK_DPD'].apply(lambda x: 1 if x >= 30 else 0)

# create feature FLAG_60DAYS_DUE where 1 = payment was late by 60 days
cc['FLAG_60DAYS_DUE'] = cc['SK_DPD'].apply(lambda x: 1 if x >= 60 else 0)

In [24]:
# for overall credit card features

credit_card_agg = {'MONTHS_BALANCE': ['min'],
                   'AMT_BALANCE': ['max', 'mean'],
                   'AMT_CREDIT_LIMIT_ACTUAL': ['max'],
                   'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum'],
                   'AMT_DRAWINGS_CURRENT': ['max', 'sum'],
                   'AMT_DRAWINGS_POS_CURRENT': ['max', 'sum'],
                   'AMT_INST_MIN_REGULARITY': ['max', 'mean'],
                   'AMT_PAYMENT_TOTAL_CURRENT': ['max', 'mean', 'sum', 'std'],
                   'AMT_TOTAL_RECEIVABLE': ['max', 'mean'],
                   'CNT_DRAWINGS_ATM_CURRENT': ['max', 'mean', 'sum'],
                   'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
                   'CNT_DRAWINGS_POS_CURRENT': ['max', 'mean', 'sum'],
                   'SK_DPD': ['max', 'mean', 'sum'],
                   'SK_DPD_DEF': ['max', 'sum'],
                   'USAGE_RATIO': ['max', 'mean'],
                   'PAYMENT_RATE': ['min', 'mean'],
                   'DRAWING_LIMIT_RATE': ['max', 'mean'],
                   'FLAG_15DAYS_DUE': ['sum'],
                   'FLAG_30DAYS_DUE': ['sum'],
                   'FLAG_60DAYS_DUE': ['sum'],
                  }

# for credit card features based on last 12, 24 and 36 months

credit_card_time_agg = {'AMT_BALANCE': ['max', 'mean'],
                        'SK_DPD': ['max', 'mean', 'sum'],
                        'USAGE_RATIO': ['max', 'mean'],
                        'FLAG_15DAYS_DUE': ['sum'],
                        'FLAG_30DAYS_DUE': ['sum'],
                        'FLAG_60DAYS_DUE': ['sum'],   
                       }

In [25]:
cc_agg = group(cc, 'CC_', credit_card_agg)

In [26]:
# looking at data in recent 1, 2 and 3 years
for months in [12, 24, 36]:
    cc_prev_id = cc[cc['MONTHS_BALANCE'] >= -months]['SK_ID_PREV'].unique()
    cc_time = cc[cc['SK_ID_PREV'].isin(cc_prev_id)]
    prefix = f'INS_{months}M_'
    cc_agg = group_and_merge(cc_time, cc_agg, prefix, credit_card_time_agg)

In [27]:
# check
cc_agg.head(3)

,SK_ID_CURR,CC_MONTHS_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_MAX,CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_SUM,CC_AMT_PAYMENT_TOTAL_CURRENT_STD,CC_AMT_TOTAL_RECEIVABLE_MAX,CC_AMT_TOTAL_RECEIVABLE_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_MAX,CC_CNT_DRAWINGS_ATM_CURRENT_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_SUM,CC_CNT_DRAWINGS_CURRENT_MAX,CC_CNT_DRAWINGS_CURRENT_MEAN,CC_CNT_DRAWINGS_CURRENT_SUM,CC_CNT_DRAWINGS_POS_CURRENT_MAX,CC_CNT_DRAWINGS_POS_CURRENT_MEAN,CC_CNT_DRAWINGS_POS_CURRENT_SUM,CC_SK_DPD_MAX,CC_SK_DPD_MEAN,CC_SK_DPD_SUM,CC_SK_DPD_DEF_MAX,CC_SK_DPD_DEF_SUM,CC_USAGE_RATIO_MAX,CC_USAGE_RATIO_MEAN,CC_PAYMENT_RATE_MIN,CC_PAYMENT_RATE_MEAN,CC_DRAWING_LIMIT_RATE_MAX,CC_DRAWING_LIMIT_RATE_MEAN,CC_FLAG_15DAYS_DUE_SUM,CC_FLAG_30DAYS_DUE_SUM,CC_FLAG_60DAYS_DUE_SUM,INS_12M_AMT_BALANCE_MAX,INS_12M_AMT_BALANCE_MEAN,INS_12M_SK_DPD_MAX,INS_12M_SK_DPD_MEAN,INS_12M_SK_DPD_SUM,INS_12M_USAGE_RATIO_MAX,INS_12M_USAGE_RATIO_MEAN,INS_12M_FLAG_15DAYS_DUE_SUM,INS_12M_FLAG_30DAYS_DUE_SUM,INS_12M_FLAG_60DAYS_DUE_SUM,INS_24M_AMT_BALANCE_MAX,INS_24M_AMT_BALANCE_MEAN,INS_24M_SK_DPD_MAX,INS_24M_SK_DPD_MEAN,INS_24M_SK_DPD_SUM,INS_24M_USAGE_RATIO_MAX,INS_24M_USAGE_RATIO_MEAN,INS_24M_FLAG_15DAYS_DUE_SUM,INS_24M_FLAG_30DAYS_DUE_SUM,INS_24M_FLAG_60DAYS_DUE_SUM,INS_36M_AMT_BALANCE_MAX,INS_36M_AMT_BALANCE_MEAN,INS_36M_SK_DPD_MAX,INS_36M_SK_DPD_MEAN,INS_36M_SK_DPD_SUM,INS_36M_USAGE_RATIO_MAX,INS_36M_USAGE_RATIO_MEAN,INS_36M_FLAG_15DAYS_DUE_SUM,INS_36M_FLAG_30DAYS_DUE_SUM,INS_36M_FLAG_60DAYS_DUE_SUM
0,100006,-6,0.000,0.000000,270000,NaN,0.0,0.00,0.000,NaN,0.000,0.000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.000000,NaN,NaN,0.0,0,0.000000,0,NaN,NaN,0.0,0,0.000000,0,0,0,0.000000,0.000000,NaN,NaN,NaN,NaN,0,0,0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
1,100011,-75,189000.000,54482.111149,180000,180000.0,180000.0,180000.00,180000.000,0.000,0.000,9000.000,3956.221849,55485.00,4520.067568,334485.000,7473.872687,189000.000,54433.179122,4.0,0.054054,4.0,4,0.054054,4,0.0,0.000000,0.0,0,0.000000,0,0,0,1.050000,0.302678,1.000000,inf,1.0000,0.013514,0,0,0,189000.000,54482.111149,0.0,0.000000,0.0,1.050000,0.302678,0.0,0.0,0.0,189000.000,54482.111149,0.0,0.000000,0.0,1.050000,0.302678,0.0,0.0,0.0,189000.000,54482.111149,0.0,0.000000,0.0,1.050000,0.302678,0.0,0.0,0.0
2,100013,-96,161420.220,18159.919219,157500,157500.0,571500.0,157500.00,571500.000,0.000,0.000,7875.000,1454.539551,153675.00,6817.172344,654448.545,21730.655260,161420.220,18101.079844,7.0,0.255556,23.0,7,0.239583,23,0.0,0.000000,0.0,1,0.010417,1,1,1,1.024890,0.115301,0.000000,inf,1.0000,0.040317,0,0,0,161420.220,18159.919219,1.0,0.010417,1.0,1.024890,0.115301,0.0,0.0,0.0,161420.220,18159.919219,1.0,0.010417,1.0,1.024890,0.115301,0.0,0.0,0.0,161420.220,18159.919219,1.0,0.010417,1.0,1.024890,0.115301,0.0,0.0,0.0
3,100021,-18,0.000,0.000000,675000,NaN,0.0,0.00,0.000,NaN,0.000,0.000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.000000,NaN,NaN,0.0,0,0.000000,0,NaN,NaN,0.0,0,0.000000,0,0,0,0.000000,0.000000,NaN,NaN,NaN,NaN,0,0,0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
4,100023,-11,0.000,0.000000,225000,NaN,0.0,0.00,0.000,NaN,0.000,0.000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.000000,NaN,NaN,0.0,0,0.000000,0,NaN,NaN,0.0,0,0.000000,0,0,0,0.000000,0.000000,NaN,NaN,NaN,NaN,0,0,0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0

## 3. Bureau & Bureau Balance
[top](#Contents)

In [28]:
missing_data(bureau_bal)

The dataframe has 3 columns.
There are 0 columns that have missing values.


,Total,% of Missing Values


In [29]:
missing_data(bureau)

The dataframe has 17 columns.
There are 7 columns that have missing values.


,Total,% of Missing Values
AMT_ANNUITY,1226791,71.5
AMT_CREDIT_MAX_OVERDUE,1124488,65.5
DAYS_ENDDATE_FACT,633653,36.9
AMT_CREDIT_SUM_LIMIT,591780,34.5
AMT_CREDIT_SUM_DEBT,257669,15.0
DAYS_CREDIT_ENDDATE,105553,6.1
AMT_CREDIT_SUM,13,0.0


In [30]:
bureau_bal

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [31]:
# set marker column to identify the row with status C
bureau_bal['marker'] = bureau_bal['STATUS'].apply(lambda x: -99 if x=='C' else random.uniform(0,100))

# drop duplicates based on subsets
bureau_bal.drop_duplicates(subset=['SK_ID_BUREAU', 'STATUS', 'marker'], keep='last', inplace=True)

# drop marker
bureau_bal.drop('marker', axis=1, inplace=True)

In [32]:
# get dummies for status column
bureau_bal = pd.get_dummies(bureau_bal, columns=['STATUS'])

In [33]:
# create feature to flag late payment for the month
bureau_bal['FLAG_DPD'] = bureau_bal.iloc[:, 3:8].sum(axis=1)

In [34]:
# check
bureau_bal.head(3)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,FLAG_DPD
8,5715448,-8,0,0,0,0,0,0,1,0,0
9,5715448,-9,1,0,0,0,0,0,0,0,0
10,5715448,-10,1,0,0,0,0,0,0,0,0
11,5715448,-11,0,0,0,0,0,0,0,1,0
12,5715448,-12,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
27299920,5041336,-47,0,0,0,0,0,0,0,1,0
27299921,5041336,-48,0,0,0,0,0,0,0,1,0
27299922,5041336,-49,0,0,0,0,0,0,0,1,0
27299923,5041336,-50,0,0,0,0,0,0,0,1,0


In [35]:
# for bureau_bal features
bureau_bal_agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size'],
                  'STATUS_0': ['mean'],
                  'STATUS_C': ['sum'],
                  'STATUS_X': ['mean'],
                  'FLAG_DPD': ['mean']
                  }

In [36]:
# merge to bureau df
bb_bal_agg = group_and_merge(bureau_bal, bureau, 'BB_BAL_', bureau_bal_agg, aggregate_by='SK_ID_BUREAU')

In [37]:
bb_bal_agg = pd.get_dummies(bb_bal_agg,  columns=['CREDIT_ACTIVE', 'CREDIT_TYPE'])

In [38]:
# check
bb_bal_agg.head(3)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BB_BAL_MONTHS_BALANCE_MIN,BB_BAL_MONTHS_BALANCE_MAX,BB_BAL_MONTHS_BALANCE_MEAN,BB_BAL_MONTHS_BALANCE_SIZE,BB_BAL_STATUS_0_MEAN,BB_BAL_STATUS_C_SUM,BB_BAL_STATUS_X_MEAN,BB_BAL_FLAG_DPD_MEAN,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,215354,5714462,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,-131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1716424,100044,5057754,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,-2493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716425,100044,5057762,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,-967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716426,246829,5057770,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,-1508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [39]:
# for DAYS_ENDDATE_FACT, this is the date (wrt credit application) the the loan ended
# null values are mainly for current active loans
# fill this with the DAYS_CREDIT_ENDDATE num if DAYS_ENDDATE_FACT is empty
bb_bal_agg['DAYS_ENDDATE_FACT'].fillna(bb_bal_agg.DAYS_CREDIT_ENDDATE, inplace=True)

# there are also rows where DAYS_CREDIT_ENDDATE is empty but DAYS_ENDDATE_FACT is not
# assumed these were closed on time at the FACT date
bb_bal_agg['DAYS_CREDIT_ENDDATE'].fillna(bb_bal_agg.DAYS_ENDDATE_FACT, inplace=True)

In [40]:
# create feature credit duration to look at length of credit
bb_bal_agg['CREDIT_DURATION'] = -bb_bal_agg['DAYS_CREDIT'] + bb_bal_agg['DAYS_CREDIT_ENDDATE']

# create feature to look at contractual end date and actual end date (indication of early payment)
bb_bal_agg['END_DATE_DIFF'] = bb_bal_agg['DAYS_CREDIT_ENDDATE'] - bb_bal_agg['DAYS_ENDDATE_FACT']

# create feature to look at debt ratio and difference
bb_bal_agg['DEBT_PERCENTAGE'] = bb_bal_agg['AMT_CREDIT_SUM'] / bb_bal_agg['AMT_CREDIT_SUM_DEBT']
bb_bal_agg['DEBT_CREDIT_DIFF'] = bb_bal_agg['AMT_CREDIT_SUM'] - bb_bal_agg['AMT_CREDIT_SUM_DEBT']
bb_bal_agg['CREDIT_TO_ANNUITY_RATIO'] = bb_bal_agg['AMT_CREDIT_SUM'] / bb_bal_agg['AMT_ANNUITY']

In [41]:
bb_bal_agg.head(2)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BB_BAL_MONTHS_BALANCE_MIN,BB_BAL_MONTHS_BALANCE_MAX,BB_BAL_MONTHS_BALANCE_MEAN,BB_BAL_MONTHS_BALANCE_SIZE,BB_BAL_STATUS_0_MEAN,BB_BAL_STATUS_C_SUM,BB_BAL_STATUS_X_MEAN,BB_BAL_FLAG_DPD_MEAN,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_DURATION,END_DATE_DIFF,DEBT_PERCENTAGE,DEBT_CREDIT_DIFF,CREDIT_TO_ANNUITY_RATIO
0,215354,5714462,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,-131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,344.0,0.0,inf,91323.0,NaN
1,215354,5714463,currency 1,-208,0,1075.0,1075.0,NaN,0,225000.0,171342.0,NaN,0.0,-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1283.0,0.0,1.313163,53658.0,NaN


In [42]:
# general bureau aggregation
bureau_aggregation = {'SK_ID_BUREAU': ['nunique'],
                      'DAYS_CREDIT': ['min', 'max', 'mean'],
                      'DAYS_CREDIT_ENDDATE': ['min', 'max'],
                      'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
                      'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
                      'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
                      'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
                      'AMT_ANNUITY': ['mean'],
                      'DEBT_CREDIT_DIFF': ['mean', 'sum'],
                      'BB_BAL_MONTHS_BALANCE_MEAN': ['mean', 'std'],
                      'BB_BAL_MONTHS_BALANCE_SIZE': ['mean', 'sum'],
                      # Categorical
                      'BB_BAL_STATUS_0_MEAN': ['mean'],
                      'BB_BAL_FLAG_DPD_MEAN': ['mean'],
                      'BB_BAL_STATUS_C_SUM': ['mean'],
                      'BB_BAL_STATUS_X_MEAN': ['mean'],
                      'CREDIT_ACTIVE_Active': ['mean'],
                      'CREDIT_ACTIVE_Closed': ['mean'],
                      'CREDIT_ACTIVE_Sold': ['mean'],
                      'CREDIT_ACTIVE_Bad debt': ['mean'],
                      'CREDIT_TYPE_Consumer credit': ['mean'],
                      'CREDIT_TYPE_Credit card': ['mean'],
                      'CREDIT_TYPE_Car loan': ['mean'],
                      'CREDIT_TYPE_Mortgage': ['mean'],
                      'CREDIT_TYPE_Microloan': ['mean'],
                     }

# recent time aggregation
bureau_time_aggregation = {'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
                           'AMT_CREDIT_SUM_OVERDUE': ['mean'],
                           'AMT_CREDIT_SUM': ['max', 'sum'],
                           'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
                           'DEBT_PERCENTAGE': ['mean'],
                           'DEBT_CREDIT_DIFF': ['mean'],
                           'BB_BAL_STATUS_0_MEAN': ['mean'],
                           'BB_BAL_FLAG_DPD_MEAN': ['mean'],
                          }

# active loans aggregation
bureau_active_aggregations = {'DAYS_CREDIT': ['max', 'mean'],
                              'DAYS_CREDIT_ENDDATE': ['min', 'max'],
                              'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
                              'AMT_CREDIT_SUM': ['max', 'sum'],
                              'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
                              'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean'],
                              'DAYS_CREDIT_UPDATE': ['min', 'mean'],
                              'DEBT_PERCENTAGE': ['mean'],
                              'DEBT_CREDIT_DIFF': ['mean'],
                              'CREDIT_TO_ANNUITY_RATIO': ['mean'],
                              'BB_BAL_MONTHS_BALANCE_MEAN': ['mean', 'std'],
                              'BB_BAL_MONTHS_BALANCE_SIZE': ['mean', 'sum'],
                             }

In [43]:
# aggregate the bureau aggregation
agg_bureau = group(bb_bal_agg, 'BUREAU_', bureau_aggregation)

In [44]:
# active loan aggregation
active = bb_bal_agg.loc[bb_bal_agg['CREDIT_ACTIVE_Active']==1]
agg_bureau = group_and_merge(active, agg_bureau, 'BUREAU_ACTIVE_', bureau_active_aggregations)

In [45]:
# time aggregations
for month in [12, 24]:
    prefix = f'BUREAU_LAST{month}M_'
    time_frame_df = bb_bal_agg[bb_bal_agg['DAYS_CREDIT'] >= -30*month]
    agg_bureau = group_and_merge(time_frame_df, agg_bureau, prefix, bureau_time_aggregation)

In [46]:
# check
agg_bureau.head(3)

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,BUREAU_AMT_CREDIT_SUM_MEAN,BUREAU_AMT_CREDIT_SUM_SUM,BUREAU_AMT_CREDIT_SUM_DEBT_MAX,BUREAU_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX,BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM,BUREAU_AMT_ANNUITY_MEAN,BUREAU_DEBT_CREDIT_DIFF_MEAN,BUREAU_DEBT_CREDIT_DIFF_SUM,BUREAU_BB_BAL_MONTHS_BALANCE_MEAN_MEAN,BUREAU_BB_BAL_MONTHS_BALANCE_MEAN_STD,BUREAU_BB_BAL_MONTHS_BALANCE_SIZE_MEAN,BUREAU_BB_BAL_MONTHS_BALANCE_SIZE_SUM,BUREAU_BB_BAL_STATUS_0_MEAN_MEAN,BUREAU_BB_BAL_FLAG_DPD_MEAN_MEAN,BUREAU_BB_BAL_STATUS_C_SUM_MEAN,BUREAU_BB_BAL_STATUS_X_MEAN_MEAN,BUREAU_CREDIT_ACTIVE_Active_MEAN,BUREAU_CREDIT_ACTIVE_Closed_MEAN,BUREAU_CREDIT_ACTIVE_Sold_MEAN,BUREAU_CREDIT_ACTIVE_Bad debt_MEAN,BUREAU_CREDIT_TYPE_Consumer credit_MEAN,BUREAU_CREDIT_TYPE_Credit card_MEAN,BUREAU_CREDIT_TYPE_Car loan_MEAN,BUREAU_CREDIT_TYPE_Mortgage_MEAN,BUREAU_CREDIT_TYPE_Microloan_MEAN,BUREAU_ACTIVE_DAYS_CREDIT_MAX,BUREAU_ACTIVE_DAYS_CREDIT_MEAN,BUREAU_ACTIVE_DAYS_CREDIT_ENDDATE_MIN,BUREAU_ACTIVE_DAYS_CREDIT_ENDDATE_MAX,BUREAU_ACTIVE_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_ACTIVE_AMT_CREDIT_SUM_MAX,BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM,BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_ACTIVE_AMT_CREDIT_SUM_OVERDUE_MAX,BUREAU_ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_ACTIVE_DAYS_CREDIT_UPDATE_MIN,BUREAU_ACTIVE_DAYS_CREDIT_UPDATE_MEAN,BUREAU_ACTIVE_DEBT_PERCENTAGE_MEAN,BUREAU_ACTIVE_DEBT_CREDIT_DIFF_MEAN,BUREAU_ACTIVE_CREDIT_TO_ANNUITY_RATIO_MEAN,BUREAU_ACTIVE_BB_BAL_MONTHS_BALANCE_MEAN_MEAN,BUREAU_ACTIVE_BB_BAL_MONTHS_BALANCE_MEAN_STD,BUREAU_ACTIVE_BB_BAL_MONTHS_BALANCE_SIZE_MEAN,BUREAU_ACTIVE_BB_BAL_MONTHS_BALANCE_SIZE_SUM,BUREAU_LAST12M_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_LAST12M_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_MAX,BUREAU_LAST12M_AMT_CREDIT_SUM_SUM,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_LAST12M_DEBT_PERCENTAGE_MEAN,BUREAU_LAST12M_DEBT_CREDIT_DIFF_MEAN,BUREAU_LAST12M_BB_BAL_STATUS_0_MEAN_MEAN,BUREAU_LAST12M_BB_BAL_FLAG_DPD_MEAN_MEAN,BUREAU_LAST24M_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_LAST24M_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_LAST24M_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_LAST24M_AMT_CREDIT_SUM_MAX,BUREAU_LAST24M_AMT_CREDIT_SUM_SUM,BUREAU_LAST24M_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_LAST24M_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_LAST24M_DEBT_PERCENTAGE_MEAN,BUREAU_LAST24M_DEBT_CREDIT_DIFF_MEAN,BUREAU_LAST24M_BB_BAL_STATUS_0_MEAN_MEAN,BUREAU_LAST24M_BB_BAL_FLAG_DPD_MEAN_MEAN
0,100001,7,-1572,-49,-735.000000,-1329.0,1778.0,NaN,NaN,378000.00,2.076236e+05,1453365.000,373239.00,85240.928571,596686.500,0.0,0.0,0.0,3545.357143,122382.642857,856678.500,-19.357143,16.020077,9.428571,66.0,0.454583,0.007519,0.571429,0.425704,0.428571,0.571429,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,-49.0,-309.333333,411.0,1778.0,NaN,NaN,378000.00,884025.000,1.988955e+05,596686.500,0.0,0.0,-16.0,-10.666667,1.841065,95779.5000,41.943113,-4.833333,4.252450,10.666667,32.0,NaN,NaN,0.0,378000.000,546345.000,241760.25,483520.5,1.269629,31412.250,0.750000,0.0,NaN,NaN,0.0,378000.0,884025.000,1.988955e+05,596686.50,1.841065,95779.5000,0.710526,0.017544
1,100002,8,-1437,-103,-874.000000,-1072.0,780.0,5043.645,1681.029,450000.00,1.081319e+05,865055.565,245781.00,49156.200000,245781.000,0.0,0.0,0.0,0.000000,98388.513000,491942.565,-22.937500,11.530821,11.625000,93.0,0.492560,0.268452,0.750000,0.168452,0.250000,0.750000,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,-103.0,-572.500000,780.0,780.0,40.500,40.5000,450000.00,481988.565,1.228905e+05,245781.000,0.0,0.0,-24.0,-15.500000,inf,118103.7825,inf,-14.250000,18.031223,9.500000,19.0,40

## 4. POS CASH
[top](#Contents)

In [47]:
missing_data(pos)

The dataframe has 8 columns.
There are 2 columns that have missing values.


,Total,% of Missing Values
CNT_INSTALMENT,1898,0.2
CNT_INSTALMENT_FUTURE,1898,0.2


In [48]:
pos.head(3)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [49]:
pos.loc[pos['SK_ID_PREV']==2116637]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
431,2116637,337030,-41,10.0,0.0,Completed,0,0
267833,2116637,337030,-43,24.0,2.0,Active,0,0


In [50]:
pos = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'])

In [51]:
# flag months with late payments
pos['FLAG_LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

In [52]:
# check
pos.head(3)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,FLAG_LATE_PAYMENT
0,1803195,182943,-31,48.0,45.0,0,0,1,0,0,0,0,0,0,0
1,1715348,367990,-33,36.0,35.0,0,0,1,0,0,0,0,0,0,0
2,1784872,397406,-32,12.0,9.0,0,0,1,0,0,0,0,0,0,0
3,1903291,269225,-35,48.0,42.0,0,0,1,0,0,0,0,0,0,0
4,2341044,334279,-35,36.0,35.0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2674109,363927,-2,48.0,39.0,0,0,1,0,0,0,0,0,0,0
1048571,1359930,438122,-4,20.0,0.0,0,0,0,0,0,1,0,0,0,0
1048572,2820160,147643,-9,24.0,15.0,0,0,1,0,0,0,0,0,0,0
1048573,1945170,307924,-11,12.0,7.0,0,0,1,0,0,0,0,0,0,0


In [53]:
pos_cash_aggregation = {'SK_ID_PREV': ['nunique'],
                        'MONTHS_BALANCE': ['min', 'max', 'size'],
                        'SK_DPD': ['max', 'mean', 'sum', 'std'],
                        'SK_DPD_DEF': ['max', 'mean', 'sum'],
                        'FLAG_LATE_PAYMENT': ['mean'],
                        'NAME_CONTRACT_STATUS_Active': ['mean'],
                        'NAME_CONTRACT_STATUS_Approved': ['mean'],
                        'NAME_CONTRACT_STATUS_Canceled': ['mean'],
                        'NAME_CONTRACT_STATUS_Completed': ['mean'],
                        'NAME_CONTRACT_STATUS_Demand': ['mean'],
                        'NAME_CONTRACT_STATUS_Returned to the store': ['mean'],
                        'NAME_CONTRACT_STATUS_Signed': ['mean'],
                       }


In [54]:
# Aggregate by SK_ID_CURR
pos_agg = group(pos, 'POS_', pos_cash_aggregation)

In [55]:
# check
pos_agg.head(3)

,SK_ID_CURR,POS_SK_ID_PREV_NUNIQUE,POS_MONTHS_BALANCE_MIN,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_SUM,POS_SK_DPD_STD,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_SK_DPD_DEF_SUM,POS_FLAG_LATE_PAYMENT_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN
0,100002,1,-13,-13,1,0,0.0,0,NaN,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,100003,2,-66,-23,2,0,0.0,0,0.0,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,100005,1,-21,-21,1,0,0.0,0,NaN,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,100007,4,-73,-3,6,0,0.0,0,0.0,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,100008,1,-19,-13,3,0,0.0,0,0.0,0,0.0,0,0.0,0.666667,0.0,0.0,0.333333,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262087,456248,2,-27,-4,8,0,0.0,0,0.0,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
262088,456249,1,-41,-40,2,0,0.0,0,0.0,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
262089,456250,2,-19,-2,5,0,0.0,0,0.0,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
262090,456253,1,-59,-59,1,0,0.0,0,NaN,0,0.0,0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [56]:
sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
gp = sort_pos.groupby('SK_ID_PREV')

In [57]:
# create empty dataframe which we will populate
df = pd.DataFrame()
df['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
df['MONTHS_BALANCE_MAX'] = gp['MONTHS_BALANCE'].max()

# Percentage of previous loans completed 
# and completed before initial term
df['POS_LOAN_COMPLETED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()
df['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INSTALMENT'].first() - gp['CNT_INSTALMENT'].last()
df['POS_COMPLETED_BEFORE_MEAN'] = df.apply(lambda x: 1 if x['POS_COMPLETED_BEFORE_MEAN'] > 0
                                                and x['POS_LOAN_COMPLETED_MEAN'] > 0 else 0, axis=1)

# Number of remaining installments (future installments) and percentage from total
df['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALMENT_FUTURE'].last()
df['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last()/gp['CNT_INSTALMENT'].last()
# Group by SK_ID_CURR and merge
df_gp = df.groupby('SK_ID_CURR').sum().reset_index()
df_gp.drop(['MONTHS_BALANCE_MAX'], axis=1, inplace= True)

pos_agg = pd.merge(pos_agg, df_gp, on= 'SK_ID_CURR', how= 'left')

In [58]:
pos_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262092 entries, 0 to 262091
Data columns (total 24 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   SK_ID_CURR                                           262092 non-null  int64  
 1   POS_SK_ID_PREV_NUNIQUE                               262092 non-null  int64  
 2   POS_MONTHS_BALANCE_MIN                               262092 non-null  int64  
 3   POS_MONTHS_BALANCE_MAX                               262092 non-null  int64  
 4   POS_MONTHS_BALANCE_SIZE                              262092 non-null  int64  
 5   POS_SK_DPD_MAX                                       262092 non-null  int64  
 6   POS_SK_DPD_MEAN                                      262092 non-null  float64
 7   POS_SK_DPD_SUM                                       262092 non-null  int64  
 8   POS_SK_DPD_STD                                       1

## 5. Previous Applications
[top](#Contents)

In [59]:
missing_data(pre_app)

The dataframe has 37 columns.
There are 16 columns that have missing values.


,Total,% of Missing Values
RATE_INTEREST_PRIMARY,1664263,99.6
RATE_INTEREST_PRIVILEGED,1664263,99.6
AMT_DOWN_PAYMENT,895844,53.6
RATE_DOWN_PAYMENT,895844,53.6
NAME_TYPE_SUITE,820405,49.1
DAYS_FIRST_DRAWING,673065,40.3
DAYS_FIRST_DUE,673065,40.3
DAYS_LAST_DUE_1ST_VERSION,673065,40.3
DAYS_LAST_DUE,673065,40.3
DAYS_TERMINATION,673065,40.3


In [60]:
# as there are duplicates based on repeat application, we filter these out
pre_app = pre_app.loc[pre_app['FLAG_LAST_APPL_PER_CONTRACT']=='Y']
pre_app = pre_app.loc[pre_app['NFLAG_LAST_APPL_IN_DAY']==1]

In [61]:
pre_app.head(3)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0


In [62]:
# create feature that aligns with what we did for the train set
# changing the process start hours to morning, afternoon or dusk
pre_app['HOUR_APPR_PROCESS_START'] = pre_app['HOUR_APPR_PROCESS_START'].apply(lambda x: 'morning' if (x>=7 and x<=12) else 'afternoon' if (x>=13 and x<=20) else 'dusk')

In [63]:
pre_app = pd.get_dummies(pre_app, columns=['NAME_CONTRACT_TYPE', 'NAME_CONTRACT_STATUS',
                                           'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 
                                           'NAME_CLIENT_TYPE', 'NAME_PRODUCT_TYPE',
                                           'CHANNEL_TYPE', 'NAME_YIELD_GROUP', 
                                           'PRODUCT_COMBINATION', 'HOUR_APPR_PROCESS_START'])

In [64]:
# Feature engineering: ratios and difference
pre_app['APPLICATION_CREDIT_DIFF'] = pre_app['AMT_APPLICATION'] - pre_app['AMT_CREDIT']
pre_app['APPLICATION_CREDIT_RATIO'] = pre_app['AMT_APPLICATION'] / pre_app['AMT_CREDIT']
pre_app['CREDIT_TO_ANNUITY_RATIO'] = pre_app['AMT_CREDIT'] / pre_app['AMT_ANNUITY']
pre_app['DOWN_PAYMENT_TO_CREDIT'] = pre_app['AMT_DOWN_PAYMENT'] / pre_app['AMT_CREDIT']

In [65]:
# Interest ratio on previous application (simplified)
total_payment = pre_app['AMT_ANNUITY'] * pre_app['CNT_PAYMENT']
pre_app['SIMPLE_INTERESTS'] = (total_payment/pre_app['AMT_CREDIT'] - 1)/pre_app['CNT_PAYMENT']

In [66]:
# Active loans - approved and not complete yet (last_due 365243)
approved = pre_app.loc[pre_app['NAME_CONTRACT_STATUS_Approved'] == 1]
active_df = approved[approved['DAYS_LAST_DUE'] == 365243]
    
# Find how much was already payed in active loans (using installments payment df)
active_pay = install_pay.loc[install_pay['SK_ID_PREV'].isin(active_df['SK_ID_PREV'])]
active_pay_agg = active_pay.groupby('SK_ID_PREV')[['AMT_INSTALMENT', 'AMT_PAYMENT']].sum()
active_pay_agg.reset_index(inplace= True)
    
# Active loans: difference of what was payed and installments
active_pay_agg['INSTALMENT_PAYMENT_DIFF'] = active_pay_agg['AMT_INSTALMENT'] - active_pay_agg['AMT_PAYMENT']

In [67]:
# Merge with active_df and calculate remaining debt amount and ratio
active_df = active_df.merge(active_pay_agg, on= 'SK_ID_PREV', how= 'left')
active_df['REMAINING_DEBT'] = active_df['AMT_CREDIT'] - active_df['AMT_PAYMENT']
active_df['REPAYMENT_RATIO'] = active_df['AMT_PAYMENT'] / active_df['AMT_CREDIT']

In [68]:
pre_active_agg = {'SK_ID_PREV': ['nunique'],
                  'SIMPLE_INTERESTS': ['mean'],
                  'AMT_ANNUITY': ['max', 'sum'],
                  'AMT_APPLICATION': ['max', 'mean'],
                  'AMT_CREDIT': ['sum'],
                  'AMT_DOWN_PAYMENT': ['max', 'mean'],
                  'DAYS_DECISION': ['min', 'mean'],
                  'CNT_PAYMENT': ['mean', 'sum'],
                  'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
                  'AMT_PAYMENT': ['sum'],
                  'INSTALMENT_PAYMENT_DIFF': ['mean', 'max'],
                  'REMAINING_DEBT': ['max', 'mean', 'sum'],
                  'REPAYMENT_RATIO': ['mean'],
                 }

In [69]:
# Perform aggregations for active applications
active_agg_df = group(active_df, 'PREV_ACTIVE_', pre_active_agg)
active_agg_df['TOTAL_REPAYMENT_RATIO'] = active_agg_df['PREV_ACTIVE_AMT_PAYMENT_SUM']/active_agg_df['PREV_ACTIVE_AMT_CREDIT_SUM']

In [70]:
# Change 365243 values to nan (missing)
pre_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

# Days last due difference (scheduled x done)
pre_app['DAYS_LAST_DUE_DIFF'] = pre_app['DAYS_LAST_DUE_1ST_VERSION'] - pre_app['DAYS_LAST_DUE']

# calculate for approved loans only too
approved['DAYS_LAST_DUE_DIFF'] = approved['DAYS_LAST_DUE_1ST_VERSION'] - approved['DAYS_LAST_DUE']

<ipython-input-70-c368b122934d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approved['DAYS_LAST_DUE_DIFF'] = approved['DAYS_LAST_DUE_1ST_VERSION'] - approved['DAYS_LAST_DUE']


In [71]:
pre_app.head(2)

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_CLIENT_TYPE_New,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell,CHANNEL_TYPE_AP+ (Cash loan),CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,HOUR_APPR_PROCESS_START_afternoon,HOUR_APPR_PROCESS_START_dusk,HOUR_APPR_PROCESS_START_morning,APPLICATION_CREDIT_DIFF,APPLICATION_CREDIT_RATIO,CREDIT_TO_ANNUITY_RATIO,DOWN_PAYMENT_TO_CREDIT,SIMPLE_INTERESTS,DAYS_LAST_DUE_DIFF
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,Y,1,0.0,0.182832,0.867336,XAP,-73,Cash through the bank,XAP,Mobile,POS,35,Connectivity,12.0,NaN,-42.0,300.0,-42.0,-37.0,0.0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0.0,1.000000,9.907942,0.0,0.017596,342.0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,Y,1,NaN,NaN,NaN,XNA,-164,XNA,XAP,XNA,Cash,-1,XNA,36.0,NaN,-134.0,916.0,NaN,NaN,1.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-72171.0,0.893815,26.983262,NaN,0.009282,NaN


In [72]:
previous_agg = {'SK_ID_PREV': ['nunique'],
                'AMT_ANNUITY': ['min', 'max', 'mean'],
                'AMT_DOWN_PAYMENT': ['max', 'mean'],
                'RATE_DOWN_PAYMENT': ['max', 'mean'],
                'DAYS_DECISION': ['min', 'max', 'mean'],
                'CNT_PAYMENT': ['max', 'mean'],
                'DAYS_TERMINATION': ['max'],
                'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
                'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean'],
                'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean', 'std'],
                'DOWN_PAYMENT_TO_CREDIT': ['mean'],
                'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
                'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
                'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
                'NAME_CONTRACT_TYPE_XNA': ['mean'],
                'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
                'NAME_CONTRACT_STATUS_Approved': ['mean'],
                'NAME_CONTRACT_STATUS_Canceled': ['mean'],
                'NAME_CONTRACT_STATUS_Refused': ['mean'],
                'NAME_CONTRACT_STATUS_Unused offer': ['mean'],
                'WEEKDAY_APPR_PROCESS_START_MONDAY': ['mean'],
                'WEEKDAY_APPR_PROCESS_START_TUESDAY': ['mean'],
                'WEEKDAY_APPR_PROCESS_START_WEDNESDAY': ['mean'],
                'WEEKDAY_APPR_PROCESS_START_THURSDAY': ['mean'],
                'WEEKDAY_APPR_PROCESS_START_FRIDAY': ['mean'],
                'WEEKDAY_APPR_PROCESS_START_SATURDAY': ['mean'],
                'WEEKDAY_APPR_PROCESS_START_SUNDAY': ['mean'],
                'NAME_TYPE_SUITE_Children': ['mean'], 
                'NAME_TYPE_SUITE_Family': ['mean'], 
                'NAME_TYPE_SUITE_Group of people': ['mean'], 
                'NAME_TYPE_SUITE_Other_A': ['mean'], 
                'NAME_TYPE_SUITE_Other_B': ['mean'], 
                'NAME_TYPE_SUITE_Spouse, partner': ['mean'], 
                'NAME_TYPE_SUITE_Unaccompanied': ['mean'], 
                'NAME_CLIENT_TYPE_New': ['mean'], 
                'NAME_CLIENT_TYPE_Refreshed': ['mean'], 
                'NAME_CLIENT_TYPE_Repeater': ['mean'], 
                'NAME_CLIENT_TYPE_XNA': ['mean'],  
                'NAME_PRODUCT_TYPE_walk-in': ['mean'],
                'NAME_PRODUCT_TYPE_x-sell': ['mean'],
                'CHANNEL_TYPE_AP+ (Cash loan)': ['mean'],
                'CHANNEL_TYPE_Car dealer': ['mean'],
                'CHANNEL_TYPE_Channel of corporate sales': ['mean'],
                'CHANNEL_TYPE_Contact center': ['mean'],
                'CHANNEL_TYPE_Credit and cash offices': ['mean'],
                'CHANNEL_TYPE_Regional / Local': ['mean'],
                'CHANNEL_TYPE_Stone': ['mean'],
                'NAME_YIELD_GROUP_XNA': ['mean'], 
                'NAME_YIELD_GROUP_high': ['mean'], 
                'NAME_YIELD_GROUP_low_action': ['mean'], 
                'NAME_YIELD_GROUP_low_normal': ['mean'], 
                'NAME_YIELD_GROUP_middle': ['mean'], 
                'PRODUCT_COMBINATION_Card Street': ['mean'],
                'PRODUCT_COMBINATION_Card X-Sell': ['mean'],
                'PRODUCT_COMBINATION_Cash': ['mean'],
                'PRODUCT_COMBINATION_Cash Street: high': ['mean'],
                'PRODUCT_COMBINATION_Cash Street: low': ['mean'],
                'PRODUCT_COMBINATION_Cash Street: middle': ['mean'],
                'PRODUCT_COMBINATION_Cash X-Sell: high': ['mean'],
                'PRODUCT_COMBINATION_Cash X-Sell: low': ['mean'],
                'PRODUCT_COMBINATION_Cash X-Sell: middle': ['mean'],
                'PRODUCT_COMBINATION_POS household with interest': ['mean'],
                'PRODUCT_COMBINATION_POS household without interest': ['mean'],
                'PRODUCT_COMBINATION_POS industry with interest': ['mean'],
                'PRODUCT_COMBINATION_POS industry without interest': ['mean'],
                'PRODUCT_COMBINATION_POS mobile with interest': ['mean'],
                'PRODUCT_COMBINATION_POS mobile without interest': ['mean'],
                'PRODUCT_COMBINATION_POS other with interest': ['mean'],
                'PRODUCT_COMBINATION_POS others without interest': ['mean'],
                'HOUR_APPR_PROCESS_START_afternoon': ['mean'],
                'HOUR_APPR_PROCESS_START_dusk': ['mean'],
                'HOUR_APPR_PROCESS_START_morning': ['mean'],
               }

In [73]:
# Perform general aggregations
agg_prev = group(pre_app, 'PREV_', previous_agg)

# Merge active loans dataframe on agg_prev
agg_prev = agg_prev.merge(active_agg_df, how='left', on='SK_ID_CURR')

In [74]:
previous_approved_agg = {'SK_ID_PREV': ['nunique'],
                         'AMT_ANNUITY': ['min', 'max', 'mean'],
                         'AMT_CREDIT': ['min', 'max', 'mean'],
                         'AMT_DOWN_PAYMENT': ['max'],
                         'AMT_GOODS_PRICE': ['max'],
                         'DAYS_DECISION': ['min', 'mean'],
                         'CNT_PAYMENT': ['max', 'mean'],
                         'DAYS_TERMINATION': ['mean'],
                         # Engineered features
                         'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
                         'APPLICATION_CREDIT_DIFF': ['max'],
                         'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
                         # only applicable for approved applications
                         'DAYS_FIRST_DRAWING': ['max', 'mean'],
                         'DAYS_FIRST_DUE': ['min', 'mean'],
                         'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
                         'DAYS_LAST_DUE': ['max', 'mean'],
                         'DAYS_LAST_DUE_DIFF': ['min', 'max', 'mean'],
                         'SIMPLE_INTERESTS': ['min', 'max', 'mean', 'std'],
                        }

previous_refused_agg = {'AMT_APPLICATION': ['max', 'mean'],
                        'AMT_CREDIT': ['min', 'max'],
                        'DAYS_DECISION': ['min', 'max', 'mean'],
                        'CNT_PAYMENT': ['max', 'mean'],
                        # Engineered features
                        'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'std'],
                        'APPLICATION_CREDIT_RATIO': ['min', 'mean'],
                        'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
                        'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
                        'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
                       }

In [75]:
# Aggregations for approved and refused loans
agg_prev = group_and_merge(approved, agg_prev, 'APPROVED_', previous_approved_agg)
refused = pre_app.loc[pre_app['NAME_CONTRACT_STATUS_Refused'] == 1]
agg_prev = group_and_merge(refused, agg_prev, 'REFUSED_', previous_refused_agg)

In [76]:
previous_time_agg = {'AMT_CREDIT': ['sum'],
                     'AMT_ANNUITY': ['mean', 'max'],
                     'SIMPLE_INTERESTS': ['mean', 'max', 'std'],
                     'DAYS_DECISION': ['min', 'mean'],
                     'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
                     # Engineered features
                     'APPLICATION_CREDIT_DIFF': ['min'],
                     'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
                     'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
                     'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
                     'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
                    }

In [77]:
# look at history for past 2  years
for time_frame in [12, 24]:
    time_frame_df = pre_app.loc[pre_app['DAYS_DECISION'] >= -30*time_frame]
    prefix = 'PREV_LAST{}M_'.format(time_frame)
    agg_prev = group_and_merge(time_frame_df, agg_prev, prefix, previous_time_agg)

## 6. Combining everything into train and test set
[top](#Contents)

In [78]:
# reading train and test datasets
train_1 = pd.read_csv('../datasets/train_1_eda.csv')
train_2 = pd.read_csv('../datasets/train_2_eda.csv')
train_3 = pd.read_csv('../datasets/train_3_eda.csv')
train_4 = pd.read_csv('../datasets/train_4_eda.csv')

test_1 = pd.read_csv('../datasets/test_1_eda.csv')
test_2 = pd.read_csv('../datasets/test_2_eda.csv')
test_3 = pd.read_csv('../datasets/test_3_eda.csv')
test_4 = pd.read_csv('../datasets/test_4_eda.csv')

train_set = [train_1, train_2, train_3, train_4]
test_set = [test_1, test_2, test_3, test_4]

In [79]:
# install_pay = install_pay_agg
# credit_card = cc_agg
# bureau = agg_bureau
# POS CASH = pos_agg
# previous_app = agg_prev

In [80]:
# create new feature that is the combination of marital status of the applicant and 
# who accomanied the applicant for the loan
for df_train in train_set:
    df_train['TYPE_SUITE_FAMILY_STATUS'] = df_train['NAME_TYPE_SUITE'] + '_' + df_train['NAME_FAMILY_STATUS']
    
for df_test in test_set:
    df_test['TYPE_SUITE_FAMILY_STATUS'] = df_test['NAME_TYPE_SUITE'] + '_' + df_test['NAME_FAMILY_STATUS']    

### 6.1. Merging past & bureau data into train_1 and test_1
`Repeat customer with bureau report`<br>
[top](#Contents)

In [81]:
print(train_1.shape)
print(test_1.shape)

(248631, 51)
(41430, 50)


In [82]:
train_1 = pd.merge(train_1, install_pay_agg, on='SK_ID_CURR', how='left')
train_1 = pd.merge(train_1, cc_agg, on='SK_ID_CURR', how='left')
train_1 = pd.merge(train_1, agg_bureau, on='SK_ID_CURR', how='left')
train_1 = pd.merge(train_1, pos_agg, on='SK_ID_CURR', how='left')
train_1 = pd.merge(train_1, agg_prev, on='SK_ID_CURR', how='left')

test_1 = pd.merge(test_1, install_pay_agg, on='SK_ID_CURR', how='left')
test_1 = pd.merge(test_1, cc_agg, on='SK_ID_CURR', how='left')
test_1 = pd.merge(test_1, agg_bureau, on='SK_ID_CURR', how='left')
test_1 = pd.merge(test_1, pos_agg, on='SK_ID_CURR', how='left')
test_1 = pd.merge(test_1, agg_prev, on='SK_ID_CURR', how='left')

In [83]:
print(train_1.shape)
print(test_1.shape)

(248631, 516)
(41430, 515)


In [84]:
# get dummies for categorical columns
train_1 = pd.get_dummies(train_1, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

test_1 = pd.get_dummies(test_1, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

In [85]:
# check
test_1.head(3)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,FLAG_DOCUMENT_16,FLAG_DOCUMENT_18,NAME_HOUSING_TYPE_CONVERT,AGE,DAYS_EMPLOYED_PENSION,YRS_EMPLOYED,HAVE_CHILDREN,LOAN_INCOME_RATIO,ANNUITY_INCOME_RATIO,CAR_BINS,FAM_MEMBERS_BIN,AVG_EXT_SOURCE,AVG_OBS_CNT_SOCIAL_CIRCLE,AVG_DEF_CNT_SOCIAL_CIRCLE,RATIO_DEF_OBS_SOCIAL_CIRCLE,INS_SK_ID_PREV_NUNIQUE,INS_DAYS_ENTRY_PAYMENT_MEAN,INS_DAYS_ENTRY_PAYMENT_MIN,INS_DAYS_ENTRY_PAYMENT_MAX,INS_AMT_INSTALMENT_MEAN,INS_AMT_INSTALMENT_MIN,INS_AMT_INSTALMENT_MAX,INS_AMT_INSTALMENT_STD,INS_AMT_PAYMENT_MEAN,INS_AMT_PAYMENT_MIN,INS_AMT_PAYMENT_MAX,INS_AMT_PAYMENT_STD,INS_DAYS_BEFORE_DUE_MEAN,INS_DAYS_BEFORE_DUE_MAX,INS_DAYS_BEFORE_DUE_STD,INS_DAYS_PAST_DUE_MEAN,INS_DAYS_PAST_DUE_MAX,INS_DAYS_PAST_DUE_STD,INS_FLAG_LATE_PAY_MEAN,INS_FLAG_LATE_PAY_SUM,INS_INSTALL_PAY_RATIO_MEAN,INS_INSTALL_PAY_RATIO_STD,INS_LATE_PAY_RATIO_MEAN,INS_LATE_PAY_RATIO_STD,INS_SIGNIFICANT_LATE_PAY_MEAN,INS_SIGNIFICANT_LATE_PAY_SUM,INS_FLAG_15DAYS_DUE_MEAN,INS_FLAG_30DAYS_DUE_MEAN,INS_FLAG_60DAYS_DUE_MEAN,INS_PAYMENT_RATIO_MEAN,INS_FLAG_PAYMENT_OVER_MEAN,INS_24M_SK_ID_PREV_SIZE,INS_24M_DAYS_ENTRY_PAYMENT_MEAN,INS_24M_DAYS_ENTRY_PAYMENT_MIN,INS_24M_DAYS_ENTRY_PAYMENT_MAX,INS_24M_AMT_INSTALMENT_MEAN,INS_24M_AMT_INSTALMENT_MIN,INS_24M_AMT_INSTALMENT_MAX,INS_24M_AMT_INSTALMENT_STD,INS_24M_AMT_PAYMENT_MEAN,INS_24M_AMT_PAYMENT_MIN,INS_24M_AMT_PAYMENT_MAX,INS_24M_AMT_PAYMENT_STD,INS_24M_DAYS_BEFORE_DUE_MEAN,INS_24M_DAYS_BEFORE_DUE_MAX,INS_24M_DAYS_BEFORE_DUE_STD,INS_24M_DAYS_PAST_DUE_MEAN,INS_24M_DAYS_PAST_DUE_MAX,INS_24M_DAYS_PAST_DUE_STD,INS_24M_FLAG_LATE_PAY_MEAN,INS_24M_FLAG_LATE_PAY_SUM,INS_24M_INSTALL_PAY_RATIO_MEAN,INS_24M_INSTALL_PAY_RATIO_STD,INS_24M_LATE_PAY_RATIO_MEAN,INS_24M_LATE_PAY_RATIO_STD,INS_24M_SIGNIFICANT_LATE_PAY_MEAN,INS_24M_SIGNIFICANT_LATE_PAY_SUM,INS_24M_FLAG_15DAYS_DUE_MEAN,INS_24M_FLAG_30DAYS_DUE_MEAN,INS_24M_FLAG_60DAYS_DUE_MEAN,INS_24M_PAYMENT_RATIO_MEAN,INS_24M_FLAG_PAYMENT_OVER_MEAN,INS_48M_SK_ID_PREV_SIZE,INS_48M_DAYS_ENTRY_PAYMENT_MEAN,INS_48M_DAYS_ENTRY_PAYMENT_MIN,INS_48M_DAYS_ENTRY_PAYMENT_MAX,INS_48M_AMT_INSTALMENT_MEAN,INS_48M_AMT_INSTALMENT_MIN,INS_48M_AMT_INSTALMENT_MAX,INS_48M_AMT_INSTALMENT_STD,INS_48M_AMT_PAYMENT_MEAN,INS_48M_AMT_PAYMENT_MIN,INS_48M_AMT_PAYMENT_MAX,INS_48M_AMT_PAYMENT_STD,INS_48M_DAYS_BEFORE_DUE_MEAN,INS_48M_DAYS_BEFORE_DUE_MAX,INS_48M_DAYS_BEFORE_DUE_STD,INS_48M_DAYS_PAST_DUE_MEAN,INS_48M_DAYS_PAST_DUE_MAX,INS_48M_DAYS_PAST_DUE_STD,INS_48M_FLAG_LATE_PAY_MEAN,INS_48M_FLAG_LATE_PAY_SUM,INS_48M_INSTALL_PAY_RATIO_MEAN,INS_48M_INSTALL_PAY_RATIO_STD,INS_48M_LATE_PAY_RATIO_MEAN,INS_48M_LATE_PAY_RATIO_STD,INS_48M_SIGNIFICANT_LATE_PAY_MEAN,INS_48M_SIGNIFICANT_LATE_PAY_SUM,INS_48M_FLAG_15DAYS_DUE_MEAN,INS_48M_FLAG_30DAYS_DUE_MEAN,INS_48M_FLAG_60DAYS_DUE_MEAN,INS_48M_PAYMENT_RATIO_MEAN,INS_48M_FLAG_PAYMENT_OVER_MEAN,CC_MONTHS_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_MAX,CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_SUM,CC_AMT_PAYMENT_TOTAL_CURRENT_STD,CC_AMT_TOTAL_RECEIVABLE_MAX,CC_AMT_TOTAL_RECEIVABLE_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_MAX,CC_CNT_DRAWINGS_ATM_CURRENT_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_SUM,CC_CNT_DRAWINGS_CURRENT_MAX,CC_CNT_DRAWINGS_CURR

In [86]:
# for some of these features, we would like to see if they were similar to the past
# i.e. if the applicant apply for the current loan in the morning, 
# did the applicant also mostly apply for loans in the morning previously?

for i in ['morning', 'afternoon', 'dusk']:
    train_1[f'APPR_PROCESS_START_DIV_{i}'] = train_1.apply(lambda x: np.nan if x[f'PROCESS_START_PERIOD_{i}']==0 
                                                              else (x[f'PROCESS_START_PERIOD_{i}'] - x[f'PREV_HOUR_APPR_PROCESS_START_{i}_MEAN']), axis=1)
    
    test_1[f'APPR_PROCESS_START_DIV_{i}'] = test_1.apply(lambda x: np.nan if x[f'PROCESS_START_PERIOD_{i}']==0 
                                                              else (x[f'PROCESS_START_PERIOD_{i}'] - x[f'PREV_HOUR_APPR_PROCESS_START_{i}_MEAN']), axis=1)
    
for j in ['Children', 'Family', 'Group of people', 'Other_A', 'Other_B', 'Spouse, partner', 'Unaccompanied']:
    train_1[f'NAME_TYPE_SUITE_DIV_{j}'] = train_1.apply(lambda x: np.nan if x[f'NAME_TYPE_SUITE_{j}']==0
                                                            else (x[f'NAME_TYPE_SUITE_{j}'] - x[f'PREV_NAME_TYPE_SUITE_{j}_MEAN']), axis=1)
    
    test_1[f'NAME_TYPE_SUITE_DIV_{j}'] = test_1.apply(lambda x: np.nan if x[f'NAME_TYPE_SUITE_{j}']==0
                                                            else (x[f'NAME_TYPE_SUITE_{j}'] - x[f'PREV_NAME_TYPE_SUITE_{j}_MEAN']), axis=1)
    
for k in ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']:
    train_1[f'WEEKDAY_APPR_PROCESS_DIV_{k}'] = train_1.apply(lambda x: np.nan if x[f'WEEKDAY_APPR_PROCESS_START_{k}']==0
                                                            else (x[f'WEEKDAY_APPR_PROCESS_START_{k}'] - x[f'PREV_WEEKDAY_APPR_PROCESS_START_{k}_MEAN']), axis=1)
    
    test_1[f'WEEKDAY_APPR_PROCESS_DIV_{k}'] = test_1.apply(lambda x: np.nan if x[f'WEEKDAY_APPR_PROCESS_START_{k}']==0
                                                            else (x[f'WEEKDAY_APPR_PROCESS_START_{k}'] - x[f'PREV_WEEKDAY_APPR_PROCESS_START_{k}_MEAN']), axis=1)

In [87]:
train_1.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,FLAG_DOCUMENT_16,FLAG_DOCUMENT_18,NAME_HOUSING_TYPE_CONVERT,AGE,DAYS_EMPLOYED_PENSION,YRS_EMPLOYED,HAVE_CHILDREN,LOAN_INCOME_RATIO,ANNUITY_INCOME_RATIO,CAR_BINS,FAM_MEMBERS_BIN,AVG_EXT_SOURCE,AVG_OBS_CNT_SOCIAL_CIRCLE,AVG_DEF_CNT_SOCIAL_CIRCLE,RATIO_DEF_OBS_SOCIAL_CIRCLE,INS_SK_ID_PREV_NUNIQUE,INS_DAYS_ENTRY_PAYMENT_MEAN,INS_DAYS_ENTRY_PAYMENT_MIN,INS_DAYS_ENTRY_PAYMENT_MAX,INS_AMT_INSTALMENT_MEAN,INS_AMT_INSTALMENT_MIN,INS_AMT_INSTALMENT_MAX,INS_AMT_INSTALMENT_STD,INS_AMT_PAYMENT_MEAN,INS_AMT_PAYMENT_MIN,INS_AMT_PAYMENT_MAX,INS_AMT_PAYMENT_STD,INS_DAYS_BEFORE_DUE_MEAN,INS_DAYS_BEFORE_DUE_MAX,INS_DAYS_BEFORE_DUE_STD,INS_DAYS_PAST_DUE_MEAN,INS_DAYS_PAST_DUE_MAX,INS_DAYS_PAST_DUE_STD,INS_FLAG_LATE_PAY_MEAN,INS_FLAG_LATE_PAY_SUM,INS_INSTALL_PAY_RATIO_MEAN,INS_INSTALL_PAY_RATIO_STD,INS_LATE_PAY_RATIO_MEAN,INS_LATE_PAY_RATIO_STD,INS_SIGNIFICANT_LATE_PAY_MEAN,INS_SIGNIFICANT_LATE_PAY_SUM,INS_FLAG_15DAYS_DUE_MEAN,INS_FLAG_30DAYS_DUE_MEAN,INS_FLAG_60DAYS_DUE_MEAN,INS_PAYMENT_RATIO_MEAN,INS_FLAG_PAYMENT_OVER_MEAN,INS_24M_SK_ID_PREV_SIZE,INS_24M_DAYS_ENTRY_PAYMENT_MEAN,INS_24M_DAYS_ENTRY_PAYMENT_MIN,INS_24M_DAYS_ENTRY_PAYMENT_MAX,INS_24M_AMT_INSTALMENT_MEAN,INS_24M_AMT_INSTALMENT_MIN,INS_24M_AMT_INSTALMENT_MAX,INS_24M_AMT_INSTALMENT_STD,INS_24M_AMT_PAYMENT_MEAN,INS_24M_AMT_PAYMENT_MIN,INS_24M_AMT_PAYMENT_MAX,INS_24M_AMT_PAYMENT_STD,INS_24M_DAYS_BEFORE_DUE_MEAN,INS_24M_DAYS_BEFORE_DUE_MAX,INS_24M_DAYS_BEFORE_DUE_STD,INS_24M_DAYS_PAST_DUE_MEAN,INS_24M_DAYS_PAST_DUE_MAX,INS_24M_DAYS_PAST_DUE_STD,INS_24M_FLAG_LATE_PAY_MEAN,INS_24M_FLAG_LATE_PAY_SUM,INS_24M_INSTALL_PAY_RATIO_MEAN,INS_24M_INSTALL_PAY_RATIO_STD,INS_24M_LATE_PAY_RATIO_MEAN,INS_24M_LATE_PAY_RATIO_STD,INS_24M_SIGNIFICANT_LATE_PAY_MEAN,INS_24M_SIGNIFICANT_LATE_PAY_SUM,INS_24M_FLAG_15DAYS_DUE_MEAN,INS_24M_FLAG_30DAYS_DUE_MEAN,INS_24M_FLAG_60DAYS_DUE_MEAN,INS_24M_PAYMENT_RATIO_MEAN,INS_24M_FLAG_PAYMENT_OVER_MEAN,INS_48M_SK_ID_PREV_SIZE,INS_48M_DAYS_ENTRY_PAYMENT_MEAN,INS_48M_DAYS_ENTRY_PAYMENT_MIN,INS_48M_DAYS_ENTRY_PAYMENT_MAX,INS_48M_AMT_INSTALMENT_MEAN,INS_48M_AMT_INSTALMENT_MIN,INS_48M_AMT_INSTALMENT_MAX,INS_48M_AMT_INSTALMENT_STD,INS_48M_AMT_PAYMENT_MEAN,INS_48M_AMT_PAYMENT_MIN,INS_48M_AMT_PAYMENT_MAX,INS_48M_AMT_PAYMENT_STD,INS_48M_DAYS_BEFORE_DUE_MEAN,INS_48M_DAYS_BEFORE_DUE_MAX,INS_48M_DAYS_BEFORE_DUE_STD,INS_48M_DAYS_PAST_DUE_MEAN,INS_48M_DAYS_PAST_DUE_MAX,INS_48M_DAYS_PAST_DUE_STD,INS_48M_FLAG_LATE_PAY_MEAN,INS_48M_FLAG_LATE_PAY_SUM,INS_48M_INSTALL_PAY_RATIO_MEAN,INS_48M_INSTALL_PAY_RATIO_STD,INS_48M_LATE_PAY_RATIO_MEAN,INS_48M_LATE_PAY_RATIO_STD,INS_48M_SIGNIFICANT_LATE_PAY_MEAN,INS_48M_SIGNIFICANT_LATE_PAY_SUM,INS_48M_FLAG_15DAYS_DUE_MEAN,INS_48M_FLAG_30DAYS_DUE_MEAN,INS_48M_FLAG_60DAYS_DUE_MEAN,INS_48M_PAYMENT_RATIO_MEAN,INS_48M_FLAG_PAYMENT_OVER_MEAN,CC_MONTHS_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_MAX,CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_SUM,CC_AMT_PAYMENT_TOTAL_CURRENT_STD,CC_AMT_TOTAL_RECEIVABLE_MAX,CC_AMT_TOTAL_RECEIVABLE_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_MAX,CC_CNT_DRAWINGS_ATM_CURRENT_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_SUM,CC_CNT_DRAWINGS_CURRENT_MAX,CC_CNT_DRAWIN

In [88]:
# identify columns not in test_1
for col in train_1:
    if col not in test_1:
        print(col)

TARGET
TYPE_SUITE_FAMILY_STATUS_Group of people_Widow
TYPE_SUITE_FAMILY_STATUS_Spouse, partner_Widow


### 6.2. Merging bureau data into train_2 and test_2
`New customer with bureau report`<br>
[top](#Contents)

In [89]:
print(train_2.shape)
print(test_2.shape)

(14859, 51)
(890, 50)


In [90]:
train_2 = pd.merge(train_2, agg_bureau, on='SK_ID_CURR', how='left')

test_2 = pd.merge(test_2, agg_bureau, on='SK_ID_CURR', how='left')

In [91]:
# get dummies for categorical columns
train_2 = pd.get_dummies(train_2, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

test_2 = pd.get_dummies(test_2, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

In [92]:
print(train_2.shape)
print(test_2.shape)

(14859, 182)
(890, 164)


In [93]:
# identify columns not in test_2
for col in train_2:
    if col not in test_2:
        print(col)

TARGET
NAME_TYPE_SUITE_Group of people
TYPE_SUITE_FAMILY_STATUS_Children_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Children_Separated
TYPE_SUITE_FAMILY_STATUS_Children_Widow
TYPE_SUITE_FAMILY_STATUS_Group of people_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Group of people_Married
TYPE_SUITE_FAMILY_STATUS_Group of people_Separated
TYPE_SUITE_FAMILY_STATUS_Group of people_Single / not married
TYPE_SUITE_FAMILY_STATUS_Other_A_Married
TYPE_SUITE_FAMILY_STATUS_Other_A_Separated
TYPE_SUITE_FAMILY_STATUS_Other_A_Single / not married
TYPE_SUITE_FAMILY_STATUS_Other_A_Widow
TYPE_SUITE_FAMILY_STATUS_Other_B_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Other_B_Single / not married
TYPE_SUITE_FAMILY_STATUS_Other_B_Widow
TYPE_SUITE_FAMILY_STATUS_Spouse, partner_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Spouse, partner_Widow


### 6.3. Merging past data into train_3 and test_3
`Repeat customer with no bureau report`<br>
[top](#Contents)

In [94]:
print(train_3.shape)
print(test_3.shape)

(41433, 51)
(6203, 50)


In [95]:
train_3 = pd.merge(train_3, install_pay_agg, on='SK_ID_CURR', how='left')
train_3 = pd.merge(train_3, cc_agg, on='SK_ID_CURR', how='left')
train_3 = pd.merge(train_3, pos_agg, on='SK_ID_CURR', how='left')
train_3 = pd.merge(train_3, agg_prev, on='SK_ID_CURR', how='left')

test_3 = pd.merge(test_3, install_pay_agg, on='SK_ID_CURR', how='left')
test_3 = pd.merge(test_3, cc_agg, on='SK_ID_CURR', how='left')
test_3 = pd.merge(test_3, pos_agg, on='SK_ID_CURR', how='left')
test_3 = pd.merge(test_3, agg_prev, on='SK_ID_CURR', how='left')

In [96]:
# get dummies for categorical columns
train_3 = pd.get_dummies(train_3, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

test_3 = pd.get_dummies(test_3, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

In [97]:
train_3.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,FLAG_DOCUMENT_16,FLAG_DOCUMENT_18,NAME_HOUSING_TYPE_CONVERT,AGE,DAYS_EMPLOYED_PENSION,YRS_EMPLOYED,HAVE_CHILDREN,LOAN_INCOME_RATIO,ANNUITY_INCOME_RATIO,CAR_BINS,FAM_MEMBERS_BIN,AVG_EXT_SOURCE,AVG_OBS_CNT_SOCIAL_CIRCLE,AVG_DEF_CNT_SOCIAL_CIRCLE,RATIO_DEF_OBS_SOCIAL_CIRCLE,INS_SK_ID_PREV_NUNIQUE,INS_DAYS_ENTRY_PAYMENT_MEAN,INS_DAYS_ENTRY_PAYMENT_MIN,INS_DAYS_ENTRY_PAYMENT_MAX,INS_AMT_INSTALMENT_MEAN,INS_AMT_INSTALMENT_MIN,INS_AMT_INSTALMENT_MAX,INS_AMT_INSTALMENT_STD,INS_AMT_PAYMENT_MEAN,INS_AMT_PAYMENT_MIN,INS_AMT_PAYMENT_MAX,INS_AMT_PAYMENT_STD,INS_DAYS_BEFORE_DUE_MEAN,INS_DAYS_BEFORE_DUE_MAX,INS_DAYS_BEFORE_DUE_STD,INS_DAYS_PAST_DUE_MEAN,INS_DAYS_PAST_DUE_MAX,INS_DAYS_PAST_DUE_STD,INS_FLAG_LATE_PAY_MEAN,INS_FLAG_LATE_PAY_SUM,INS_INSTALL_PAY_RATIO_MEAN,INS_INSTALL_PAY_RATIO_STD,INS_LATE_PAY_RATIO_MEAN,INS_LATE_PAY_RATIO_STD,INS_SIGNIFICANT_LATE_PAY_MEAN,INS_SIGNIFICANT_LATE_PAY_SUM,INS_FLAG_15DAYS_DUE_MEAN,INS_FLAG_30DAYS_DUE_MEAN,INS_FLAG_60DAYS_DUE_MEAN,INS_PAYMENT_RATIO_MEAN,INS_FLAG_PAYMENT_OVER_MEAN,INS_24M_SK_ID_PREV_SIZE,INS_24M_DAYS_ENTRY_PAYMENT_MEAN,INS_24M_DAYS_ENTRY_PAYMENT_MIN,INS_24M_DAYS_ENTRY_PAYMENT_MAX,INS_24M_AMT_INSTALMENT_MEAN,INS_24M_AMT_INSTALMENT_MIN,INS_24M_AMT_INSTALMENT_MAX,INS_24M_AMT_INSTALMENT_STD,INS_24M_AMT_PAYMENT_MEAN,INS_24M_AMT_PAYMENT_MIN,INS_24M_AMT_PAYMENT_MAX,INS_24M_AMT_PAYMENT_STD,INS_24M_DAYS_BEFORE_DUE_MEAN,INS_24M_DAYS_BEFORE_DUE_MAX,INS_24M_DAYS_BEFORE_DUE_STD,INS_24M_DAYS_PAST_DUE_MEAN,INS_24M_DAYS_PAST_DUE_MAX,INS_24M_DAYS_PAST_DUE_STD,INS_24M_FLAG_LATE_PAY_MEAN,INS_24M_FLAG_LATE_PAY_SUM,INS_24M_INSTALL_PAY_RATIO_MEAN,INS_24M_INSTALL_PAY_RATIO_STD,INS_24M_LATE_PAY_RATIO_MEAN,INS_24M_LATE_PAY_RATIO_STD,INS_24M_SIGNIFICANT_LATE_PAY_MEAN,INS_24M_SIGNIFICANT_LATE_PAY_SUM,INS_24M_FLAG_15DAYS_DUE_MEAN,INS_24M_FLAG_30DAYS_DUE_MEAN,INS_24M_FLAG_60DAYS_DUE_MEAN,INS_24M_PAYMENT_RATIO_MEAN,INS_24M_FLAG_PAYMENT_OVER_MEAN,INS_48M_SK_ID_PREV_SIZE,INS_48M_DAYS_ENTRY_PAYMENT_MEAN,INS_48M_DAYS_ENTRY_PAYMENT_MIN,INS_48M_DAYS_ENTRY_PAYMENT_MAX,INS_48M_AMT_INSTALMENT_MEAN,INS_48M_AMT_INSTALMENT_MIN,INS_48M_AMT_INSTALMENT_MAX,INS_48M_AMT_INSTALMENT_STD,INS_48M_AMT_PAYMENT_MEAN,INS_48M_AMT_PAYMENT_MIN,INS_48M_AMT_PAYMENT_MAX,INS_48M_AMT_PAYMENT_STD,INS_48M_DAYS_BEFORE_DUE_MEAN,INS_48M_DAYS_BEFORE_DUE_MAX,INS_48M_DAYS_BEFORE_DUE_STD,INS_48M_DAYS_PAST_DUE_MEAN,INS_48M_DAYS_PAST_DUE_MAX,INS_48M_DAYS_PAST_DUE_STD,INS_48M_FLAG_LATE_PAY_MEAN,INS_48M_FLAG_LATE_PAY_SUM,INS_48M_INSTALL_PAY_RATIO_MEAN,INS_48M_INSTALL_PAY_RATIO_STD,INS_48M_LATE_PAY_RATIO_MEAN,INS_48M_LATE_PAY_RATIO_STD,INS_48M_SIGNIFICANT_LATE_PAY_MEAN,INS_48M_SIGNIFICANT_LATE_PAY_SUM,INS_48M_FLAG_15DAYS_DUE_MEAN,INS_48M_FLAG_30DAYS_DUE_MEAN,INS_48M_FLAG_60DAYS_DUE_MEAN,INS_48M_PAYMENT_RATIO_MEAN,INS_48M_FLAG_PAYMENT_OVER_MEAN,CC_MONTHS_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_MAX,CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CC_AMT_PAYMENT_TOTAL_CURRENT_SUM,CC_AMT_PAYMENT_TOTAL_CURRENT_STD,CC_AMT_TOTAL_RECEIVABLE_MAX,CC_AMT_TOTAL_RECEIVABLE_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_MAX,CC_CNT_DRAWINGS_ATM_CURRENT_MEAN,CC_CNT_DRAWINGS_ATM_CURRENT_SUM,CC_CNT_DRAWINGS_CURRENT_MAX,CC_CNT_DRAWIN

In [98]:
for i in ['morning', 'afternoon', 'dusk']:
    train_3[f'APPR_PROCESS_START_DIV_{i}'] = train_3.apply(lambda x: np.nan if x[f'PROCESS_START_PERIOD_{i}']==0 
                                                              else (x[f'PROCESS_START_PERIOD_{i}'] - x[f'PREV_HOUR_APPR_PROCESS_START_{i}_MEAN']), axis=1)
    
    test_3[f'APPR_PROCESS_START_DIV_{i}'] = test_3.apply(lambda x: np.nan if x[f'PROCESS_START_PERIOD_{i}']==0 
                                                              else (x[f'PROCESS_START_PERIOD_{i}'] - x[f'PREV_HOUR_APPR_PROCESS_START_{i}_MEAN']), axis=1)
    
for j in ['Children', 'Family', 'Group of people', 'Other_A', 'Other_B', 'Spouse, partner', 'Unaccompanied']:
    train_3[f'NAME_TYPE_SUITE_DIV_{j}'] = train_3.apply(lambda x: np.nan if x[f'NAME_TYPE_SUITE_{j}']==0
                                                            else (x[f'NAME_TYPE_SUITE_{j}'] - x[f'PREV_NAME_TYPE_SUITE_{j}_MEAN']), axis=1)
    
    test_3[f'NAME_TYPE_SUITE_DIV_{j}'] = test_3.apply(lambda x: np.nan if x[f'NAME_TYPE_SUITE_{j}']==0
                                                            else (x[f'NAME_TYPE_SUITE_{j}'] - x[f'PREV_NAME_TYPE_SUITE_{j}_MEAN']), axis=1)
    
for k in ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']:
    train_3[f'WEEKDAY_APPR_PROCESS_DIV_{k}'] = train_3.apply(lambda x: np.nan if x[f'WEEKDAY_APPR_PROCESS_START_{k}']==0
                                                            else (x[f'WEEKDAY_APPR_PROCESS_START_{k}'] - x[f'PREV_WEEKDAY_APPR_PROCESS_START_{k}_MEAN']), axis=1)
    
    test_3[f'WEEKDAY_APPR_PROCESS_DIV_{k}'] = test_3.apply(lambda x: np.nan if x[f'WEEKDAY_APPR_PROCESS_START_{k}']==0
                                                            else (x[f'WEEKDAY_APPR_PROCESS_START_{k}'] - x[f'PREV_WEEKDAY_APPR_PROCESS_START_{k}_MEAN']), axis=1)

In [99]:
print(train_3.shape)
print(test_3.shape)

(41433, 505)
(6203, 498)


In [100]:
# identify columns not in test_3
for col in train_3:
    if col not in test_3:
        print(col)

TARGET
TYPE_SUITE_FAMILY_STATUS_Group of people_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Group of people_Separated
TYPE_SUITE_FAMILY_STATUS_Group of people_Widow
TYPE_SUITE_FAMILY_STATUS_Other_A_Separated
TYPE_SUITE_FAMILY_STATUS_Other_A_Widow
TYPE_SUITE_FAMILY_STATUS_Spouse, partner_Widow


### 6.4. Preparing data for train_4 and test_4
`New customer with no bureau report`<br>
[top](#Contents)

In [101]:
print(train_4.shape)
print(test_4.shape)

(2587, 51)
(221, 50)


In [102]:
# get dummies for categorical columns
train_4 = pd.get_dummies(train_4, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

test_4 = pd.get_dummies(test_4, columns=['NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_FAMILY_STATUS', 'TYPE_SUITE_FAMILY_STATUS', 'PROCESS_START_PERIOD'])

In [103]:
print(train_4.shape)
print(test_4.shape)

(2587, 96)
(221, 77)


In [104]:
# identify columns not in test_4
for col in train_4:
    if col not in test_4:
        print(col)

TARGET
NAME_TYPE_SUITE_Children
NAME_TYPE_SUITE_Other_B
TYPE_SUITE_FAMILY_STATUS_Children_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Children_Married
TYPE_SUITE_FAMILY_STATUS_Children_Single / not married
TYPE_SUITE_FAMILY_STATUS_Children_Widow
TYPE_SUITE_FAMILY_STATUS_Family_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Family_Separated
TYPE_SUITE_FAMILY_STATUS_Group of people_Widow
TYPE_SUITE_FAMILY_STATUS_Other_A_Married
TYPE_SUITE_FAMILY_STATUS_Other_A_Single / not married
TYPE_SUITE_FAMILY_STATUS_Other_B_Civil marriage
TYPE_SUITE_FAMILY_STATUS_Other_B_Married
TYPE_SUITE_FAMILY_STATUS_Other_B_Separated
TYPE_SUITE_FAMILY_STATUS_Other_B_Single / not married
TYPE_SUITE_FAMILY_STATUS_Other_B_Widow
TYPE_SUITE_FAMILY_STATUS_Spouse, partner_Separated
TYPE_SUITE_FAMILY_STATUS_Spouse, partner_Single / not married


### 6.5. Identifying alternative features and saving files for modeling
[top](#Contents)

In [105]:
# rename column name so that it can be read into the lightgb model
train_1 = train_1.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 
train_2 = train_2.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 
train_3 = train_3.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 
train_4 = train_4.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 

test_1 = test_1.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 
test_2 = test_2.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 
test_3 = test_3.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 
test_4 = test_4.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x)) 

In [106]:
# for group 1:

to_drop_for_train_1 = [
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Widow',
]

alternative_feat_1 = [
    'DAYS_ID_PUBLISH',
    'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE',
    'FLAG_CONT_MOBILE', 
    'FLAG_PHONE',
    'REGION_RATING_CLIENT_W_CITY',
    'REG_REGION_NOT_LIVE_REGION',
    'REG_REGION_NOT_WORK_REGION',
    'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY',
    'OBS_30_CNT_SOCIAL_CIRCLE', 
    'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE', 
    'DAYS_LAST_PHONE_CHANGE', 
    'CAR_BINS',
    'FAM_MEMBERS_BIN', 
    'AVG_OBS_CNT_SOCIAL_CIRCLE', 
    'AVG_DEF_CNT_SOCIAL_CIRCLE', 
    'RATIO_DEF_OBS_SOCIAL_CIRCLE',
    'NAME_TYPE_SUITE_Children',
    'NAME_TYPE_SUITE_Family',
    'NAME_TYPE_SUITE_Group_of_people',
    'NAME_TYPE_SUITE_Other_A',
    'NAME_TYPE_SUITE_Other_B',
    'NAME_TYPE_SUITE_Spouse_partner',
    'NAME_TYPE_SUITE_Unaccompanied',
    'WEEKDAY_APPR_PROCESS_START_MONDAY',
    'WEEKDAY_APPR_PROCESS_START_TUESDAY',
    'WEEKDAY_APPR_PROCESS_START_WEDNESDAY',
    'WEEKDAY_APPR_PROCESS_START_THURSDAY',
    'WEEKDAY_APPR_PROCESS_START_FRIDAY',
    'WEEKDAY_APPR_PROCESS_START_SATURDAY',
    'WEEKDAY_APPR_PROCESS_START_SUNDAY',
    'NAME_FAMILY_STATUS_Civil_marriage',
    'NAME_FAMILY_STATUS_Married',
    'NAME_FAMILY_STATUS_Separated',
    'NAME_FAMILY_STATUS_Single_not_married',
    'NAME_FAMILY_STATUS_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Children_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Children_Married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Children_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Family_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Family_Married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Family_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Married',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Widow', 
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Married',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Widow',
    'PROCESS_START_PERIOD_afternoon',
    'PROCESS_START_PERIOD_dusk',
    'PROCESS_START_PERIOD_morning',    
    'PREV_WEEKDAY_APPR_PROCESS_START_MONDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_TUESDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_THURSDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_FRIDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_SATURDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_SUNDAY_MEAN',
    'PREV_NAME_TYPE_SUITE_Children_MEAN',
    'PREV_NAME_TYPE_SUITE_Family_MEAN',
    'PREV_NAME_TYPE_SUITE_Group_of_people_MEAN',
    'PREV_NAME_TYPE_SUITE_Other_A_MEAN',
    'PREV_NAME_TYPE_SUITE_Other_B_MEAN',
    'PREV_NAME_TYPE_SUITE_Spouse_partner_MEAN',
    'PREV_NAME_TYPE_SUITE_Unaccompanied_MEAN',
    'PREV_HOUR_APPR_PROCESS_START_morning_MEAN',
    'PREV_HOUR_APPR_PROCESS_START_afternoon_MEAN',
    'PREV_HOUR_APPR_PROCESS_START_dusk_MEAN',
    'APPR_PROCESS_START_DIV_morning',
    'APPR_PROCESS_START_DIV_afternoon',
    'APPR_PROCESS_START_DIV_dusk',
    'NAME_TYPE_SUITE_DIV_Children', 
    'NAME_TYPE_SUITE_DIV_Family',
    'NAME_TYPE_SUITE_DIV_Group_of_people',
    'NAME_TYPE_SUITE_DIV_Other_A',
    'NAME_TYPE_SUITE_DIV_Other_B',
    'NAME_TYPE_SUITE_DIV_Spouse_partner',
    'NAME_TYPE_SUITE_DIV_Unaccompanied',
    'WEEKDAY_APPR_PROCESS_DIV_MONDAY',
    'WEEKDAY_APPR_PROCESS_DIV_TUESDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_WEDNESDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_THURSDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_FRIDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_SATURDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_SUNDAY',   
]

In [107]:
# for group 2:

to_drop_for_train_2 = [
    'NAME_TYPE_SUITE_Group_of_people',
    'TYPE_SUITE_FAMILY_STATUS_Children_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Children_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Children_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Married',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Widow',
]

alternative_feat_2 = [
    'DAYS_ID_PUBLISH',
    'FLAG_EMP_PHONE', 
    'FLAG_WORK_PHONE', 
    'FLAG_CONT_MOBILE', 
    'FLAG_PHONE',
    'REGION_RATING_CLIENT_W_CITY',
    'REG_REGION_NOT_LIVE_REGION',
    'REG_REGION_NOT_WORK_REGION',
    'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY',
    'OBS_30_CNT_SOCIAL_CIRCLE', 
    'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE', 
    'DAYS_LAST_PHONE_CHANGE', 
    'CAR_BINS',
    'FAM_MEMBERS_BIN', 
    'AVG_OBS_CNT_SOCIAL_CIRCLE', 
    'AVG_DEF_CNT_SOCIAL_CIRCLE', 
    'RATIO_DEF_OBS_SOCIAL_CIRCLE',
    'NAME_TYPE_SUITE_Children',
    'NAME_TYPE_SUITE_Family',
    'NAME_TYPE_SUITE_Group_of_people',
    'NAME_TYPE_SUITE_Other_A',
    'NAME_TYPE_SUITE_Other_B',
    'NAME_TYPE_SUITE_Spouse_partner',
    'NAME_TYPE_SUITE_Unaccompanied',
    'WEEKDAY_APPR_PROCESS_START_MONDAY',
    'WEEKDAY_APPR_PROCESS_START_TUESDAY',
    'WEEKDAY_APPR_PROCESS_START_WEDNESDAY',
    'WEEKDAY_APPR_PROCESS_START_THURSDAY',
    'WEEKDAY_APPR_PROCESS_START_FRIDAY',
    'WEEKDAY_APPR_PROCESS_START_SATURDAY',
    'WEEKDAY_APPR_PROCESS_START_SUNDAY',
    'NAME_FAMILY_STATUS_Civil_marriage',
    'NAME_FAMILY_STATUS_Married',
    'NAME_FAMILY_STATUS_Separated',
    'NAME_FAMILY_STATUS_Single_not_married',
    'NAME_FAMILY_STATUS_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Children_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Children_Married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Children_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Family_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Family_Married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Family_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Married',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Married',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Widow',
    'PROCESS_START_PERIOD_afternoon',
    'PROCESS_START_PERIOD_dusk',
    'PROCESS_START_PERIOD_morning',  
]

In [108]:
# for group 3:

to_drop_for_train_3 = [
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Widow',
]

alternative_feat_3 = [
    'DAYS_ID_PUBLISH',
    'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE',
    'FLAG_CONT_MOBILE', 
    'FLAG_PHONE',
    'REGION_RATING_CLIENT_W_CITY',
    'REG_REGION_NOT_LIVE_REGION',
    'REG_REGION_NOT_WORK_REGION',
    'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY',
    'OBS_30_CNT_SOCIAL_CIRCLE', 
    'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE', 
    'DAYS_LAST_PHONE_CHANGE', 
    'CAR_BINS',
    'FAM_MEMBERS_BIN', 
    'AVG_OBS_CNT_SOCIAL_CIRCLE', 
    'AVG_DEF_CNT_SOCIAL_CIRCLE', 
    'RATIO_DEF_OBS_SOCIAL_CIRCLE',
    'NAME_TYPE_SUITE_Children',
    'NAME_TYPE_SUITE_Family',
    'NAME_TYPE_SUITE_Group_of_people',
    'NAME_TYPE_SUITE_Other_A',
    'NAME_TYPE_SUITE_Other_B',
    'NAME_TYPE_SUITE_Spouse_partner',
    'NAME_TYPE_SUITE_Unaccompanied',
    'WEEKDAY_APPR_PROCESS_START_MONDAY',
    'WEEKDAY_APPR_PROCESS_START_TUESDAY',
    'WEEKDAY_APPR_PROCESS_START_WEDNESDAY',
    'WEEKDAY_APPR_PROCESS_START_THURSDAY',
    'WEEKDAY_APPR_PROCESS_START_FRIDAY',
    'WEEKDAY_APPR_PROCESS_START_SATURDAY',
    'WEEKDAY_APPR_PROCESS_START_SUNDAY',
    'NAME_FAMILY_STATUS_Civil_marriage',
    'NAME_FAMILY_STATUS_Married',
    'NAME_FAMILY_STATUS_Separated',
    'NAME_FAMILY_STATUS_Single_not_married',
    'NAME_FAMILY_STATUS_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Children_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Children_Married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Children_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Family_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Family_Married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Family_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Married',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Widow', 
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Married',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Widow',
    'PROCESS_START_PERIOD_afternoon',
    'PROCESS_START_PERIOD_dusk',
    'PROCESS_START_PERIOD_morning',    
    'PREV_WEEKDAY_APPR_PROCESS_START_MONDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_TUESDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_THURSDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_FRIDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_SATURDAY_MEAN',
    'PREV_WEEKDAY_APPR_PROCESS_START_SUNDAY_MEAN',
    'PREV_NAME_TYPE_SUITE_Children_MEAN',
    'PREV_NAME_TYPE_SUITE_Family_MEAN',
    'PREV_NAME_TYPE_SUITE_Group_of_people_MEAN',
    'PREV_NAME_TYPE_SUITE_Other_A_MEAN',
    'PREV_NAME_TYPE_SUITE_Other_B_MEAN',
    'PREV_NAME_TYPE_SUITE_Spouse_partner_MEAN',
    'PREV_NAME_TYPE_SUITE_Unaccompanied_MEAN',
    'PREV_HOUR_APPR_PROCESS_START_morning_MEAN',
    'PREV_HOUR_APPR_PROCESS_START_afternoon_MEAN',
    'PREV_HOUR_APPR_PROCESS_START_dusk_MEAN',
    'APPR_PROCESS_START_DIV_morning',
    'APPR_PROCESS_START_DIV_afternoon',
    'APPR_PROCESS_START_DIV_dusk',
    'NAME_TYPE_SUITE_DIV_Children', 
    'NAME_TYPE_SUITE_DIV_Family',
    'NAME_TYPE_SUITE_DIV_Group_of_people',
    'NAME_TYPE_SUITE_DIV_Other_A',
    'NAME_TYPE_SUITE_DIV_Other_B',
    'NAME_TYPE_SUITE_DIV_Spouse_partner',
    'NAME_TYPE_SUITE_DIV_Unaccompanied',
    'WEEKDAY_APPR_PROCESS_DIV_MONDAY',
    'WEEKDAY_APPR_PROCESS_DIV_TUESDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_WEDNESDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_THURSDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_FRIDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_SATURDAY', 
    'WEEKDAY_APPR_PROCESS_DIV_SUNDAY',   
]

In [109]:
# for group 4:

to_drop_for_train_4 = [
    'NAME_TYPE_SUITE_Children',
    'NAME_TYPE_SUITE_Other_B',
    'TYPE_SUITE_FAMILY_STATUS_Children_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Children_Married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Family_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Family_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Single_not_married',    
]

alternative_feat_4 = [
    'DAYS_ID_PUBLISH',
    'FLAG_EMP_PHONE', 
    'FLAG_WORK_PHONE', 
    'FLAG_CONT_MOBILE', 
    'FLAG_PHONE',
    'REGION_RATING_CLIENT_W_CITY',
    'REG_REGION_NOT_LIVE_REGION',
    'REG_REGION_NOT_WORK_REGION',
    'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY',
    'OBS_30_CNT_SOCIAL_CIRCLE', 
    'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE', 
    'DAYS_LAST_PHONE_CHANGE', 
    'CAR_BINS',
    'FAM_MEMBERS_BIN', 
    'AVG_OBS_CNT_SOCIAL_CIRCLE', 
    'AVG_DEF_CNT_SOCIAL_CIRCLE', 
    'RATIO_DEF_OBS_SOCIAL_CIRCLE',
    'NAME_TYPE_SUITE_Children',
    'NAME_TYPE_SUITE_Family',
    'NAME_TYPE_SUITE_Group_of_people',
    'NAME_TYPE_SUITE_Other_A',
    'NAME_TYPE_SUITE_Other_B',
    'NAME_TYPE_SUITE_Spouse_partner',
    'NAME_TYPE_SUITE_Unaccompanied',
    'WEEKDAY_APPR_PROCESS_START_MONDAY',
    'WEEKDAY_APPR_PROCESS_START_TUESDAY',
    'WEEKDAY_APPR_PROCESS_START_WEDNESDAY',
    'WEEKDAY_APPR_PROCESS_START_THURSDAY',
    'WEEKDAY_APPR_PROCESS_START_FRIDAY',
    'WEEKDAY_APPR_PROCESS_START_SATURDAY',
    'WEEKDAY_APPR_PROCESS_START_SUNDAY',
    'NAME_FAMILY_STATUS_Civil_marriage',
    'NAME_FAMILY_STATUS_Married',
    'NAME_FAMILY_STATUS_Separated',
    'NAME_FAMILY_STATUS_Single_not_married',
    'NAME_FAMILY_STATUS_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Children_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Children_Married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Children_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Family_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Family_Married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Family_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Family_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Married',
    'TYPE_SUITE_FAMILY_STATUS_Group_of_people_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_A_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Other_B_Widow',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Married',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Spouse_partner_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Civil_marriage',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Separated',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Single_not_married',
    'TYPE_SUITE_FAMILY_STATUS_Unaccompanied_Widow',
    'PROCESS_START_PERIOD_afternoon',
    'PROCESS_START_PERIOD_dusk',
    'PROCESS_START_PERIOD_morning',   
]

In [110]:
# save individual train and test files
train_1.to_csv('../datasets/train_1_final.csv', index=False)
train_2.to_csv('../datasets/train_2_final.csv', index=False)
train_3.to_csv('../datasets/train_3_final.csv', index=False)
train_4.to_csv('../datasets/train_4_final.csv', index=False)

test_1.to_csv('../datasets/test_1_final.csv', index=False)
test_2.to_csv('../datasets/test_2_final.csv', index=False)
test_3.to_csv('../datasets/test_3_final.csv', index=False)
test_4.to_csv('../datasets/test_4_final.csv', index=False)